In [1]:
import os
import re
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, mapping, LineString, MultiLineString, shape
import rasterio
from rasterio.mask import mask as rio_mask
from rasterio.enums import Resampling
from rasterio.features import shapes
from rasterio.warp import transform
import numpy as np
from pathlib import Path
from pyproj import Transformer
import glob
from shapely.ops import unary_union
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing
import warnings
import matplotlib.pyplot as plt
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import random
from scipy.stats import pearsonr, spearmanr
import seaborn as sns
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import math
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
os.chdir("..")

In [ ]:
os.getcwd()

PATHS

In [3]:
# Regressions
regress_path = pd.read_csv(os.path.join(os.getcwd(), "3_output/Global_22_07_25/B73_huber_reg.csv"))#/dark_fr_030/8_bits/S3_15Obs_100W/jul_3_25/Po_River_huber_reg_3.csv")) #No_Norm/dark_fr_030/8_bits/S3_15Obs_100W/jun_6_25/
                                        #"Po_River_huber_reg.csv"))
poly_path = gpd.read_file(os.path.join(os.getcwd(), "2_intermediate/Store/Binary_Masks/Cape_fear/Shps/Cape_cx.shp"))
tif_path = os.path.join(os.getcwd(), "0_data/External/Bathymetries/namerica/NC/cape_fear_egm2008.tif")
base_folder = os.path.join(os.getcwd(),"2_intermediate/Store/Binary_Masks/Cape_fear/")

/var/folders/fg/t6lktxg91c5b91pyb0rk6x080000gn/T/ipykernel_23721/850668533.py:2: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  regress_path = pd.read_csv(os.path.join(os.getcwd(), "3_output/Global_22_07_25/B73_huber_reg.csv"))#/dark_fr_030/8_bits/S3_15Obs_100W/jul_3_25/Po_River_huber_reg_3.csv")) #No_Norm/dark_fr_030/8_bits/S3_15Obs_100W/jun_6_25/


#### 1. Subsetting only the nodes lying inside the raster 

In [4]:
with rasterio.open(tif_path) as src:
    nod = src.nodata
    # pull lon/lat from your DataFrame
    xs = regress_path["lon"].values
    ys = regress_path["lat"].values

    # ─── NEW: reproject point coords into the raster’s CRS ───
    xs, ys = transform(
        "EPSG:4326",    # source CRS of your lon/lat
        src.crs,        # target CRS (raster.crs)
        xs.tolist(),
        ys.tolist(),
    )
    xs = np.array(xs)
    ys = np.array(ys)

    # Check raster bounds (now in raster CRS)
    left, bottom, right, top = src.bounds
    inside_bounds = (xs >= left) & (xs <= right) & (ys >= bottom) & (ys <= top)
    
    # Preallocate array for sampled values (nan for out-of-bounds)
    sampled = np.full(xs.shape, np.nan)
    xs_in = xs[inside_bounds]
    ys_in = ys[inside_bounds]
    idxs_in = np.where(inside_bounds)[0]
    if len(xs_in) > 0:
        vals = list(src.sample(zip(xs_in, ys_in)))
        vals = np.array([v[0] for v in vals])
        sampled[idxs_in] = vals

    # Mask: inside bounds AND not nodata AND not nan
    mask = inside_bounds & (~np.isnan(sampled)) & ((nod is None) | (sampled != nod))

# subset and export
regress_path_inside = regress_path[mask]
print(f"{regress_path_inside.shape[0]} nodes inside raster, {regress_path.shape[0]} total nodes")

regress_path_inside.to_csv(
    os.path.join(os.getcwd(),
        "2_intermediate/Store/Validation/csv/Cape_fear/Cape_fear_validate_nodes.csv"
    ),
    index=False
)

1209 nodes inside raster, 236234 total nodes


#### Extract from bathymetry the bottom of the river channel

In [ ]:
with rasterio.open(tif_path) as src:
    # 1) show me the raster’s CRS
    print("Raster CRS is:", src.crs)

    # 2) if your poly_path is still EPSG:4326, move it into the raster’s CRS
    if poly_path.crs != src.crs:
        poly_path = poly_path.to_crs(src.crs)

    results = []
    for _, row in poly_path.iterrows():
        node = row["node_id"]
        geom = [mapping(row.geometry)]
        out_image, _ = rio_mask(
            src,
            geom,
            crop=True,         # keeps memory use tiny
            all_touched=True,
            nodata=src.nodata,
            filled=False
        )
        arr = out_image[0].astype("float32")
        arr = np.ma.masked_equal(arr, src.nodata)
        arr = np.ma.masked_where(np.isnan(arr), arr)
        min_val = arr.min() if arr.count() > 0 else np.nan
        results.append({"node_id": node, "min_value": float(min_val)})

lprb = pd.DataFrame(results)
lprb.to_csv(
    os.path.join(
        os.getcwd(),
        "2_intermediate/Store/Validation/csv/Garonne/Garonne_channel_min_values.csv"
    ),
    index=False
)
    

In [ ]:
lprb = pd.read_csv(os.path.join(os.getcwd(), "2_intermediate/Store/Validation/csv/Po/River_channel_min_values.csv"))

Merge the two csv files, the one with the regression values and the one with the minimum values in the river bed, and then  
estimate the difference and errors between the intercepts and the real bathymetry

In [ ]:
# Ensure node_id is string in both DataFrames for a valid merge
regress_path["node_id"] = regress_path["node_id"].astype(str)
lprb["node_id"] = lprb["node_id"].astype(str)

merged = regress_path.merge(
    lprb[["node_id", "min_value"]],
    on="node_id",
    how="inner"
)

intercept = np.where(
    merged["intercept2"].isnull(),
    merged["intercept1"],
    merged["intercept2"]
)

merged["error"] = intercept - merged["min_value"]
merged["abs_error"] = merged["error"].abs()
merged["rel_error_%"] = merged["error"] / merged["min_value"] * 100

merged.to_csv(os.path.join(os.getcwd(), "2_intermediate/Store/Validation/csv/Garonne/Garonne_reg_bottom_error.csv"), index=False)

In [ ]:
master_csv_path = os.path.join(os.getcwd(), "2_intermediate/Store/Validation/csv/Garonne/Garonne_reg_bottom_error.csv")

Create folders for each node_id

In [ ]:
# 2) Base directory where you want to create the folders
#    For example: "/path/to/output_folders"


# 3) Read the master CSV into a DataFrame
master_df = pd.read_csv(master_csv_path)

# 4) Extract unique node_id values (as strings)
unique_nodes = master_df["node_id"].astype(str).unique()

# 5) Write the list of node_id codes to a CSV (no header, one per line)
list_csv_path = os.path.join(base_folder, "node_ids_list.csv")
pd.Series(unique_nodes).to_csv(list_csv_path, index=False, header=False)

# 6) Loop over each unique node_id and create a folder if it doesn't already exist
for node_id in unique_nodes:
    node_folder = os.path.join(base_folder, node_id)
    # exist_ok=True means “do nothing if the folder already exists”
    os.makedirs(node_folder, exist_ok=True)
    print(f"Ensured folder exists: {node_folder}")

Create in each node_id folder, a csv file with a list of WSE extracted from the regression csv for each node_id

In [ ]:
def export_wse_csvs(
    master_csv_path: str,
    base_folder: str,
    subtract_value: float = 2.04,
    apply_subtraction: bool = False
):
    """
    For each node_id in master_df, write:
        1) raw WSEs → base_folder/node_id/node_id.csv
        2) (optionally) adjusted WSEs (WSE - subtract_value)
            → base_folder/node_id/node_id_adjusted.csv

    Args:
        master_csv_path: path to CSV with columns 'node_id' and 'wse'
        base_folder: directory that will contain subfolders named by node_id
        subtract_value: the value to subtract from each WSE when apply_subtraction=True
        apply_subtraction: if True, also create the adjusted CSV
    """
    # Check if file exists
    if not os.path.exists(master_csv_path):
        print(f"Error: File '{master_csv_path}' does not exist.")
        return

    # Read the master table
    df = pd.read_csv(master_csv_path)

    # Loop through each distinct node_id
    for node_id in df["node_id"].astype(str).unique():
        # Grab all the WSE values for this node
        wse_values = df.loc[df["node_id"].astype(str) == node_id, "wse"]

        # Ensure the node's folder exists
        node_folder = os.path.join(base_folder, node_id)
        os.makedirs(node_folder, exist_ok=True)

        # 1) Write the raw WSEs
        raw_path = os.path.join(node_folder, f"{node_id}.csv")
        wse_values.to_csv(raw_path, index=False, header=False)
        print(f"✔ Saved {len(wse_values)} raw WSEs to {raw_path}")

        # 2) If requested, write the adjusted WSEs
        if apply_subtraction:
            adjusted = wse_values - subtract_value
            adj_path = os.path.join(node_folder, f"{node_id}_adjusted.csv")
            adjusted.to_csv(adj_path, index=False, header=False)
            print(f"✔ Saved {len(adjusted)} adjusted WSEs to {adj_path}")


In [ ]:
# Just export raw WSE lists:
export_wse_csvs(master_csv_path, base_folder)

# Export raw WSE lists *and* create adjusted CSVs (subtract 2.04):
#export_wse_csvs(master_csv_path, base_folder, apply_subtraction=True)

# Export with a different subtraction value, say 1.5:
#export_wse_csvs(master_csv_path, base_folder, subtract_value=1.5, apply_subtraction=True)

Create a mask for each WSE in each node_id

In [ ]:
# --- SETTINGS ---

# (Optional) If you only want to process a subset of node_ids, list them here.
# Otherwise set `limit_to` = None to process every subfolder under `base_folder`.
limit_to = None
# Example to process only two nodes:
# limit_to = ["21406100080691", "21406100080702"]


# === STEP 1: Read the DEM ONCE ===

with rasterio.open(tif_path) as src:
    # Read the full DEM array at native resolution
    dem = src.read(1, resampling=Resampling.nearest)
    profile = src.profile.copy()
    nodata_value = src.nodata if src.nodata is not None else -9999

# Update the profile for writing uint8 masks with 1=water, 0=non-water, 255=nodata
profile.update({
    "dtype": rasterio.uint8,
    "count": 1,
    "nodata": 255,
    "compress": "lzw"
})

# === STEP 2: Locate all node_id subfolders under `base_folder` ===

# Use glob to find all directories directly under `base_folder`
node_folders = [
    d for d in glob.glob(os.path.join(base_folder, "*"))
    if os.path.isdir(d)
]

# If user specified a subset, filter to only those names
if limit_to is not None:
    node_folders = [
        d for d in node_folders
        if os.path.basename(d) in limit_to
    ]

print(f"🔍 Found {len(node_folders)} node‐folders to process.\n")


# === STEP 3: Loop over each node_folder ===

for node_folder in node_folders:
    node_id = os.path.basename(node_folder)
    print(f" Processing node_id = {node_id}")

    # 3a) Find all CSV files in this folder
    csv_files = glob.glob(os.path.join(node_folder, "*.csv"))
    if not csv_files:
        print(f" No CSVs found inside {node_folder}. Skipping.\n")
        continue

    # 3b) For each CSV, read its WSE values (single‐column, no header)
    for csv_path in csv_files:
        # We assume each CSV has one column with no header (just WSE floats/ints).
        try:
            df_wse = pd.read_csv(csv_path, header=None, names=["wse"], dtype=float)
        except Exception as e:
            print(f"Failed to read {csv_path}: {e}")
            continue

        wse_values = df_wse["wse"].dropna().values
        if wse_values.size == 0:
            print(f"No valid WSE values in {csv_path}. Skipping this file.")
            continue

        print(f"Found {len(wse_values)} thresholds in {os.path.basename(csv_path)}")

        # 3c) Create an output subfolder for these masks, if desired.
        #     You can write masks back into node_folder, or into a subfolder:
        masks_folder = os.path.join(node_folder, "masks")
        os.makedirs(masks_folder, exist_ok=True)

        # 3d) For each WSE threshold, build a binary mask
        for t in wse_values:
            # Build a safe filename: replace dot with underscore, and strip any minus sign
            t_str = str(t).replace(".", "_").replace("-", "m")
            out_name = f"mask_{t_str}.tif"
            out_path = os.path.join(masks_folder, out_name)

            # If the mask already exists, skip (do not overwrite)
            if os.path.exists(out_path):
                print(f"      • {out_name} already exists. Skipping.")
                continue

            # Build the mask: water where DEM ≤ t, non-water otherwise
            # First, initialize all to 0 (non-water)
            mask = np.zeros_like(dem, dtype=np.uint8)

            # Where DEM is valid and DEM ≤ threshold → set to 1 (water)
            valid = dem != nodata_value
            mask[np.logical_and(valid, dem <= t)] = 1

            # Where DEM is nodata → set to 255
            mask[dem == nodata_value] = 255

            # Write out the mask TIFF
            with rasterio.open(out_path, "w", **profile) as dst:
                dst.write(mask, 1)

            print(f"Saved {out_name}")

    print("")  # blank line before next node

print("All node‐folders processed.")


In [ ]:
limit_to    = None                     # or e.g. ["21406100080691", "21406100080702"]

# Tunable: number of worker threads for writing masks
_max_workers = min(8, os.cpu_count() or 4)

def _write_mask_single(out_path, profile, dem, valid, nodata_mask, nodata_value, t):
    """Build and write one mask TIFF for threshold t."""
    mask = np.zeros_like(dem, dtype=np.uint8)
    np.less_equal(dem, t, where=valid, out=mask, casting="unsafe")
    mask[nodata_mask] = 255
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(mask, 1)
    return out_path

with rasterio.Env(GDAL_NUM_THREADS="ALL_CPUS", VSI_CACHE="TRUE", VSI_CACHE_SIZE=str(256*1024*1024)):
    # === STEP 1: Read the DEM ONCE ===
    with rasterio.open(tif_path) as src:
        dem = src.read(1, resampling=Resampling.nearest)
        profile = src.profile.copy()
        nodata_value = src.nodata if src.nodata is not None else -9999

    profile.update({
        "dtype": rasterio.uint8,
        "count": 1,
        "nodata": 255,
        "compress": "lzw"
    })

    nodata_mask = (dem == nodata_value)
    valid = ~nodata_mask

    # === STEP 2: Locate all node_id subfolders ===
    node_folders = [d for d in glob.glob(os.path.join(base_folder, "*")) if os.path.isdir(d)]
    if limit_to is not None:
        node_folders = [d for d in node_folders if os.path.basename(d) in limit_to]

    print(f"🔍 Found {len(node_folders)} node‐folders to process.\n")

    # === STEP 3: Process each folder ===
    for node_folder in node_folders:
        node_id = os.path.basename(node_folder)
        print(f" Processing node_id = {node_id}")

        csv_files = glob.glob(os.path.join(node_folder, "*.csv"))
        if not csv_files:
            print(f" No CSVs found inside {node_folder}. Skipping.\n")
            continue

        for csv_path in csv_files:
            try:
                df_wse = pd.read_csv(csv_path, header=None, names=["wse"], dtype=float)
            except Exception as e:
                print(f"Failed to read {csv_path}: {e}")
                continue

            wse_values = df_wse["wse"].dropna().values
            if wse_values.size == 0:
                print(f"No valid WSE values in {csv_path}. Skipping this file.")
                continue

            print(f"Found {len(wse_values)} thresholds in {os.path.basename(csv_path)}")

            masks_folder = os.path.join(node_folder, "masks")
            os.makedirs(masks_folder, exist_ok=True)

            futures = []
            with ThreadPoolExecutor(max_workers=_max_workers) as ex:
                for t in wse_values:
                    t_str = str(t).replace(".", "_").replace("-", "m")
                    out_name = f"mask_{t_str}.tif"
                    out_path = os.path.join(masks_folder, out_name)

                    if os.path.exists(out_path):
                        print(f"      • {out_name} already exists. Skipping.")
                        continue

                    futures.append(
                        ex.submit(_write_mask_single, out_path, profile, dem, valid, nodata_mask, nodata_value, t)
                    )

                for f in as_completed(futures):
                    try:
                        done_path = f.result()
                        print(f"Saved {os.path.basename(done_path)}")
                    except Exception as e:
                        print(f"      • Failed to write a mask: {e}")

        print("")  # blank line

    print("All node‐folders processed.")

#### Measuring river widths

In [ ]:
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

def ensure_mask_georeferenced(mask_path, ref_crs, ref_transform):
    with rasterio.open(mask_path, 'r+') as dst:
        needs_update = False
        if dst.crs is None:
            dst.crs = ref_crs
            needs_update = True
        if dst.transform.is_identity:
            dst.transform = ref_transform
            needs_update = True
        if needs_update:
            print(f"Updated georeferencing for: {os.path.basename(mask_path)}")

def fix_all_masks_in_base_folder(base_folder, ref_crs, ref_transform):
    for node_folder in os.listdir(base_folder):
        masks_dir = os.path.join(base_folder, node_folder, "masks")
        if os.path.isdir(masks_dir):
            for mask_path in glob.glob(os.path.join(masks_dir, "mask_*.tif")):
                ensure_mask_georeferenced(mask_path, ref_crs, ref_transform)

def extract_wse(mask_filename):
    # strip extension and split on underscores
    name = os.path.splitext(os.path.basename(mask_filename))[0]
    parts = name.split('_')   # ["mask", "50", "535"]
    if len(parts) == 3:
        return float(f"{parts[1]}.{parts[2]}")  # "50" + "." + "535" → 50.535
    return None

def possible_node_id_strings(node_id):
    candidates = set()
    candidates.add(str(node_id))
    try:
        as_float = float(node_id)
        candidates.add(str(int(as_float)))
        candidates.add(f"{as_float:.3f}")
        candidates.add(f"{as_float:.1f}")
    except Exception:
        pass
    return list(candidates)

def measure_mask_segments_by_sampling(polyline, mask_path, gdf_crs, spacing=1):
    with rasterio.open(mask_path) as src:
        raster_crs = src.crs

        # ─── REPLACED: use the mask/raster CRS (e.g. EPSG:6339) instead of Italy UTM ───
        utm_crs = raster_crs

        if gdf_crs and gdf_crs != utm_crs:
            polyline_proj = gpd.GeoSeries([polyline], crs=gdf_crs).to_crs(utm_crs).iloc[0]
        else:
            polyline_proj = polyline

        length = polyline_proj.length
        if length < spacing:
            return []

        num_points = int(length / spacing)
        distances = np.linspace(0, length, num_points)
        points = [polyline_proj.interpolate(d) for d in distances]

        coords = gpd.GeoSeries(points, crs=utm_crs).to_crs(raster_crs)
        coords = [(pt.x, pt.y) for pt in coords]

        vals = [v[0] for v in src.sample(coords)]
        vals = [int(round(v)) if v in [0, 1] else 0 for v in vals]

        segments = []
        in_segment = False
        start_dist = None

        for i in range(1, len(vals)):
            prev, curr = vals[i - 1], vals[i]
            if not in_segment and prev == 0 and curr == 1:
                in_segment = True
                start_dist = distances[i]
            elif in_segment and prev == 1 and curr == 0:
                end_dist = distances[i]
                seg_len = end_dist - start_dist
                if seg_len >= 20:
                    segments.append(seg_len)
                in_segment = False

        return segments

def process_single_polyline(args):
    idx, row, base_folder, node_id_field, gdf_crs = args
    node_id_raw = row[node_id_field]
    possible_names = possible_node_id_strings(node_id_raw)
    folder_path = None
    used_node_id = None
    for node_id in possible_names:
        test_path = os.path.join(base_folder, node_id, 'masks')
        if os.path.exists(test_path):
            folder_path = test_path
            used_node_id = node_id
            break
    if folder_path is None:
        return []

    polyline = row.geometry
    mask_files = sorted(glob.glob(os.path.join(folder_path, 'mask_*.tif')))
    if not mask_files:
        return []

    row_results = []
    max_segments = 0
    for mask_file in mask_files:
        wse_val = extract_wse(os.path.basename(mask_file))
        if wse_val is None:
            continue
        segment_lengths = measure_mask_segments_by_sampling(polyline, mask_file, gdf_crs)
        if segment_lengths:
            width = max(segment_lengths)
            n_segments = len(segment_lengths)
            others = sorted([s for s in segment_lengths if s != width], reverse=True)
        else:
            width = 0.0
            n_segments = 0
            others = []
        max_segments = max(max_segments, len(others))
        row_results.append([used_node_id, wse_val, width, n_segments] + others)
    return row_results, max_segments

def process_all_parallel(base_folder, shapefile_path, output_csv_path, reference_raster_path, node_id_field='node_id', n_workers=None):
    gdf = gpd.read_file(shapefile_path)
    all_results = []
    max_segments_overall = 0

    with rasterio.open(reference_raster_path) as ref:
        ref_crs = ref.crs
        ref_transform = ref.transform

    print("Checking and fixing georeferencing for all masks...")
    fix_all_masks_in_base_folder(base_folder, ref_crs, ref_transform)
    print("Georeference check complete.\n")

    if n_workers is None:
        n_workers = multiprocessing.cpu_count()

    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        args_iterable = [
            (idx, row, base_folder, node_id_field, gdf.crs)
            for idx, row in gdf.iterrows()
        ]
        futures = [executor.submit(process_single_polyline, args) for args in args_iterable]
        for i, future in enumerate(as_completed(futures), 1):
            res = future.result()
            if res:
                row_results, max_segs = res
                all_results.extend(row_results)
                max_segments_overall = max(max_segments_overall, max_segs)
            if i % 5 == 0 or i == len(futures):
                print(f"Processed {i}/{len(futures)} polylines...")

    columns = ['node_id', 'wse', 'width', 'n_segments'] + [f'seg_{i+1}' for i in range(max_segments_overall)]
    padded_results = []
    for row in all_results:
        base = row[:4]
        segs = row[4:]
        segs = segs + [float('nan')] * (max_segments_overall - len(segs))
        padded_results.append(base + segs)

    df = pd.DataFrame(padded_results, columns=columns)
    df.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")


# Example usage (edit paths as needed):
path2poly = os.path.join(os.getcwd(), "2_intermediate/Store/Binary_Masks/Garonne/Shps/garonne_cx.shp")
process_all_parallel(base_folder, path2poly, os.path.join(os.getcwd(),"2_intermediate/Store/Binary_Masks/Garonne/Bath_widths.csv"), 
                    tif_path, n_workers=10)

Alternative code to measure widths manually

In [ ]:
def measure_mask_widths_for_polyline(polyline, mask_path):
    with rasterio.open(mask_path) as src:
        arr = src.read(1)
        transform = src.transform
        crs = src.crs
        mask_polys = [shape(geom) for geom, val in shapes(arr, mask=(arr == 1), transform=transform)]
        if not mask_polys:
            raise RuntimeError("No area with value=1 found in mask.")
        from shapely.ops import unary_union
        mask_union = unary_union(mask_polys)
        # Reproject polyline if needed
        if polyline.crs and polyline.crs != crs:
            line = polyline.to_crs(crs).geometry.iloc[0]
        else:
            line = polyline.geometry.iloc[0]
        clipped = line.intersection(mask_union)
        if clipped.is_empty:
            raise RuntimeError("Polyline does not cross the mask '1' area.")
        # Collect all segment lengths
        if isinstance(clipped, LineString):
            segments = [clipped]
        elif isinstance(clipped, MultiLineString):
            segments = list(clipped.geoms)
        else:
            raise RuntimeError("Unexpected geometry after intersection.")
        segment_lengths = [seg.length for seg in segments]
        return segment_lengths

# --- User inputs ---
mask_path = os.path.join(os.getcwd(), "2_intermediate/Store/Binary_Masks/Atrato/61100201100491/masks/mask_24_29153.tif")
#polyline_path = '/path/to/Atrato.shp'
node_id_field = 'node_id'      # Change to your field name if different
node_id_value = 61100201100491.0         # The node_id you want

# Select polyline by node_id
gdf = gpd.read_file(path2poly)
# Robust matching: allow for int/float/string representations
def node_id_match(x):
    try:
        return float(x) == float(node_id_value)
    except Exception:
        return False

selected = gdf[gdf[node_id_field].apply(node_id_match)]
print("Matching node_ids:", selected[node_id_field].tolist())
if len(selected) != 1:
    raise ValueError(f"node_id {node_id_value} not found or not unique in {path2poly}.")

segment_lengths = measure_mask_widths_for_polyline(selected, mask_path)

df = pd.DataFrame({
    'node_id': [node_id_value]*len(segment_lengths),
    'segment': range(1, len(segment_lengths)+1),
    'width': segment_lengths
})

print(df)

Calculate slopes from Huber simple and piecewise regressions

One

In [ ]:
def fit_huber(X_input, y_input):
    """
    Fit a simple Huber linear model to (X_input, y_input).
    Returns the trained model, predictions, and residuals.
    """
    model = HuberRegressor(epsilon=1.345, max_iter=500)
    X_reshaped = X_input.reshape(-1, 1)
    model.fit(X_reshaped, y_input)
    y_pred = model.predict(X_reshaped)
    residuals = y_input - y_pred
    return model, y_pred, residuals

def fit_regressions_for_group(X, y):
    """
    Always does a single-segment Huber fit.
    Returns a dict with keys:
    - 'bath_slope1': the slope
    - 'bath_slope2': None
    - 'r2': R² score
    - 'type': 'simple'
    - 'intercepts': (intercept, None)
    - 'bp': None
    """
    model_s, y_pred_s, res_s = fit_huber(X, y)
    slope1 = model_s.coef_[0]
    intercept1 = model_s.intercept_
    r2 = r2_score(y, y_pred_s)
    return {
        'bath_slope1': slope1,
        'bath_slope2': None,
        'r2': r2,
        'type': 'simple',
        'intercepts': (intercept1, None),
        'bp': None
    }

def groupwise_regression(csv_path):
    """
    Read CSV with columns ['node_id','width','wse'], fit Huber per node_id group.
    Returns (result_df, group_results).
    """
    df = pd.read_csv(csv_path)
    result_rows = []
    group_results = {}
    for node_id, group in df.groupby('node_id'):
        group = group.dropna(subset=['width', 'wse'])
        if len(group) < 3:
            continue
        X = group['width'].values
        y = group['wse'].values
        res = fit_regressions_for_group(X, y)
        result_rows.append({
            'node_id':     node_id,
            'bath_slope1': res['bath_slope1'],
            'bath_slope2': res['bath_slope2'],
            'r2':          res['r2'],
            'type':        res['type']
        })
        group_results[node_id] = (group, res)
    result_df = pd.DataFrame(result_rows)
    return result_df, group_results

def plot_random_regressions(group_results, n=8):
    """
    Scatter each group's (width, wse) and overlay the Huber fit line.
    """
    plot_ids = random.sample(list(group_results.keys()), min(n, len(group_results)))
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))
    axs = axs.flatten()

    for i, node_id in enumerate(plot_ids):
        group, res = group_results[node_id]
        X = group['width'].values
        y = group['wse'].values
        ax = axs[i]

        ax.scatter(X, y, label="Data", alpha=0.6)
        slope = res['bath_slope1']
        intercept = res['intercepts'][0]
        x_plot = np.linspace(X.min(), X.max(), 100)
        y_fit = slope * x_plot + intercept
        ax.plot(x_plot, y_fit, linewidth=2, label=f"Simple: slope={slope:.2f}")

        ax.set_title(f"node_id={node_id}\nR²={res['r2']:.2f}")
        ax.set_xlabel("Width")
        ax.set_ylabel("WSE")
        ax.legend()
        ax.grid(True)

    # Hide unused axes
    for ax in axs[len(plot_ids):]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()

path2w_wse = os.path.join(os.getcwd(), "2_intermediate/Store/Binary_Masks/Cape_fear/Bath_widths.csv")

Two

In [ ]:
def compute_aic(rss, n, k):
    return n * np.log(rss / n) + 2 * k

def fit_huber(X_input, y_input):
    model = HuberRegressor(epsilon=1.345, max_iter=500)
    model.fit(X_input, y_input)
    y_pred = model.predict(X_input)
    residuals = y_input - y_pred
    return model, y_pred, residuals

def fit_regressions_for_group(X, y):
    n = len(X)
    results = {'best_model_type': 'simple', 'bath_slope1': None, 'bath_slope2': None, 'r2': None}
    model_s, y_pred_s, res_s = fit_huber(X.reshape(-1, 1), y)
    rss_s = np.sum(res_s ** 2)
    aic_s = compute_aic(rss_s, n, 2)
    best_model_type = 'simple'
    best_aic = aic_s
    slope1 = model_s.coef_[0]
    intercept1 = model_s.intercept_
    slope2 = intercept2 = None
    best_bp = None

    unique_x = np.unique(X)
    if len(unique_x) >= 3:
        for bp in unique_x[1:-1]:
            left_n = np.sum(X < bp)
            right_n = np.sum(X >= bp)
            if 10 <= n <= 15 and (left_n < 4 or right_n < 4):
                continue
            if n >= 16 and (left_n < 0.3 * n or right_n < 0.3 * n):
                continue
            X_pw = np.column_stack([X, np.maximum(0, X - bp)])
            try:
                model_pw, y_pred_pw, res_pw = fit_huber(X_pw, y)
            except:
                continue
            h0, h1, delta = model_pw.intercept_, model_pw.coef_[0], model_pw.coef_[1]
            if abs(delta) < 0.08:
                continue
            slope_left = h1
            intercept_left = h0
            slope_right = h1 + delta
            intercept_right = h0 - delta * bp
            if slope_left <= 0 or slope_right <= 0:
                continue
            rss_pw = np.sum(res_pw ** 2)
            aic_pw = compute_aic(rss_pw, n, 3)
            if aic_pw < best_aic:
                best_model_type = 'piecewise'
                best_aic = aic_pw
                slope1, intercept1 = slope_right, intercept_right
                slope2, intercept2 = slope_left, intercept_left
                best_bp = bp

    # bath_slope1 = higher widths (right segment), bath_slope2 = lower widths (left segment)
    if best_model_type == 'piecewise':
        if slope1 is not None and slope2 is not None:
            results['bath_slope1'] = slope1
            results['bath_slope2'] = slope2
            X_pw = np.column_stack([X, np.maximum(0, X - best_bp)])
            y_pred = slope2 * X + intercept2
            y_pred[X >= best_bp] = slope1 * X[X >= best_bp] + intercept1
            results['r2'] = r2_score(y, y_pred)
        else:
            results['bath_slope1'] = slope1
            results['bath_slope2'] = None
            results['r2'] = r2_score(y, model_s.predict(X.reshape(-1, 1)))
    else:
        results['bath_slope1'] = slope1
        results['bath_slope2'] = None
        results['r2'] = r2_score(y, model_s.predict(X.reshape(-1, 1)))
    results['type'] = best_model_type
    results['intercepts'] = (intercept1, intercept2)
    results['bp'] = best_bp
    return results

def groupwise_regression(csv_path):
    df = pd.read_csv(csv_path)
    result_rows = []
    group_results = {}
    for node_id, group in df.groupby('node_id'):
        group = group.dropna(subset=['width', 'wse'])
        if len(group) < 3:
            continue
        X = group['width'].values
        y = group['wse'].values
        res = fit_regressions_for_group(X, y)
        result_rows.append({
            'node_id': node_id,
            'bath_slope1': res['bath_slope1'],
            'bath_slope2': res['bath_slope2'],
            'r2': res['r2'],
            'type': res['type']
        })
        group_results[node_id] = (group, res)
    result_df = pd.DataFrame(result_rows)
    return result_df, group_results

def plot_random_regressions(group_results, n=8):
    plot_ids = random.sample(list(group_results.keys()), min(n, len(group_results)))
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))
    axs = axs.flatten()
    for i, node_id in enumerate(plot_ids):
        group, res = group_results[node_id]
        X = group['width'].values
        y = group['wse'].values
        ax = axs[i]
        ax.scatter(X, y, label="Data", color='steelblue')
        x_plot = np.linspace(X.min(), X.max(), 100)
        if res['type'] == 'simple':
            slope, intercept = res['bath_slope1'], res['intercepts'][0]
            y_fit = slope * x_plot + intercept
            ax.plot(x_plot, y_fit, color='crimson', linewidth=2, label=f"Simple: slope={slope:.2f}")
        else:
            slope1, intercept1 = res['bath_slope1'], res['intercepts'][0]
            slope2, intercept2 = res['bath_slope2'], res['intercepts'][1]
            bp = res['bp']
            y_left = slope2 * x_plot + intercept2
            y_right = slope1 * x_plot + intercept1
            ax.plot(x_plot[x_plot < bp], y_left[x_plot < bp], color='crimson', linewidth=2, label=f"Left: slope={slope2:.2f}")
            ax.plot(x_plot[x_plot >= bp], y_right[x_plot >= bp], color='crimson', linewidth=2, linestyle='--', label=f"Right: slope={slope1:.2f}")
            ax.axvline(bp, color='gray', linestyle=':', label=f"Breakpoint = {bp:.2f}")
        ax.set_title(f"node_id={node_id}\nR2={res['r2']:.2f}")
        ax.set_xlabel("Width")
        ax.set_ylabel("WSE")
        ax.legend()
        ax.grid(True)
    plt.tight_layout()
    plt.show()
path2w_wse = os.path.join(os.getcwd(), "2_intermediate/Store/Binary_Masks/Cape_fear/Bath_widths.csv")

Three

In [ ]:
def compute_aic(rss, n, k):
    return n * np.log(rss / n) + 2 * k

def fit_huber(X_input, y_input):
    model = HuberRegressor(epsilon=1.345, max_iter=500)
    model.fit(X_input, y_input)
    y_pred = model.predict(X_input)
    residuals = y_input - y_pred
    return model, y_pred, residuals

def fit_regressions_for_group(X, y, min_width_range=40):
    n = len(X)
    results = {
        'best_model_type': 'simple',
        'bath_slope1': None,
        'bath_slope2': None,
        'bath_slope3': None,
        'r2': None,
        'intercepts': (None, None, None),
        'bp': None
    }

    def is_invalid_slope(s):
        # invalid if None/NaN/inf or < 0.001 or negative
        return (s is None) or np.isnan(s) or np.isinf(s) or (s < 0.001)

    # === Simple model ===
    model_s, y_pred_s, res_s = fit_huber(X.reshape(-1, 1), y)
    rss_s = np.sum(res_s ** 2)
    aic_s = compute_aic(rss_s, n, 2)
    slope_simple = model_s.coef_[0]
    intercept_simple = model_s.intercept_
    best_model_type = 'simple'
    best_aic = aic_s

    slope1 = slope2 = slope3 = None
    intercept1 = intercept2 = intercept3 = None
    best_bp = None
    best_bps = None

    unique_x = np.unique(X)

    # === Try 2 breakpoints (three segments) ===
    if len(unique_x) >= 4:
        for i in range(1, len(unique_x) - 2):
            for j in range(i + 1, len(unique_x) - 1):
                bp1 = unique_x[i]
                bp2 = unique_x[j]
                if bp1 >= bp2:
                    continue

                seg1 = X[X < bp1]
                seg2 = X[(X >= bp1) & (X < bp2)]
                seg3 = X[X >= bp2]

                # need at least 2 points per side and sufficient width span
                if any(seg.size < 2 for seg in (seg1, seg2, seg3)):
                    continue
                if any(np.ptp(seg) < min_width_range for seg in (seg1, seg2, seg3)):
                    continue

                X_pw2 = np.column_stack([X, np.maximum(0, X - bp1), np.maximum(0, X - bp2)])
                try:
                    model_pw2, y_pred_pw2, res_pw2 = fit_huber(X_pw2, y)
                except Exception:
                    continue

                h0 = model_pw2.intercept_
                h1, d1, d2 = model_pw2.coef_

                slope_left  = h1
                slope_mid   = h1 + d1
                slope_right = h1 + d1 + d2

                # reject invalid / negative / too-flat segments
                if any(is_invalid_slope(s) for s in (slope_left, slope_mid, slope_right)):
                    continue

                intercept_left  = h0
                intercept_mid   = h0 - d1 * bp1
                intercept_right = h0 - d1 * bp1 - d2 * bp2

                rss_pw2 = np.sum(res_pw2 ** 2)
                aic_pw2 = compute_aic(rss_pw2, n, 4)  # 4 params: h0, h1, d1, d2

                if aic_pw2 < best_aic:
                    best_model_type = 'piecewise_2bp'
                    best_aic = aic_pw2
                    # keep naming convention: slope1 = rightmost (higher widths)
                    slope1, slope2, slope3 = slope_right, slope_mid, slope_left
                    intercept1, intercept2, intercept3 = intercept_right, intercept_mid, intercept_left
                    best_bps = (bp1, bp2)

    # === Try 1 breakpoint (two segments) ===
    if best_model_type == 'simple' and len(unique_x) >= 3:
        for bp in unique_x[1:-1]:
            seg_left = X[X < bp]
            seg_right = X[X >= bp]

            if seg_left.size < 2 or seg_right.size < 2:
                continue
            if (np.ptp(seg_left) < min_width_range) or (np.ptp(seg_right) < min_width_range):
                continue

            X_pw1 = np.column_stack([X, np.maximum(0, X - bp)])
            try:
                model_pw1, y_pred_pw1, res_pw1 = fit_huber(X_pw1, y)
            except Exception:
                continue

            h0 = model_pw1.intercept_
            h1, d1 = model_pw1.coef_

            slope_left  = h1
            slope_right = h1 + d1

            if any(is_invalid_slope(s) for s in (slope_left, slope_right)):
                continue

            intercept_left  = h0
            intercept_right = h0 - d1 * bp

            rss_pw1 = np.sum(res_pw1 ** 2)
            aic_pw1 = compute_aic(rss_pw1, n, 3)  # 3 params: h0, h1, d1

            if aic_pw1 < best_aic:
                best_model_type = 'piecewise'
                best_aic = aic_pw1
                # naming convention: slope1 = right (higher widths), slope2 = left
                slope1, slope2 = slope_right, slope_left
                intercept1, intercept2 = intercept_right, intercept_left
                best_bp = bp

    # === Store results (match original keys) ===
    if best_model_type == 'piecewise_2bp':
        results['type'] = 'piecewise_2bp'
        results['bath_slope1'] = slope1
        results['bath_slope2'] = slope2
        results['bath_slope3'] = slope3
        results['intercepts'] = (intercept1, intercept2, intercept3)
        results['bp'] = best_bps
        y_pred = np.empty_like(y)
        y_pred[:] = slope3 * X + intercept3
        y_pred[X >= best_bps[0]] = slope2 * X[X >= best_bps[0]] + intercept2
        y_pred[X >= best_bps[1]] = slope1 * X[X >= best_bps[1]] + intercept1
        results['r2'] = r2_score(y, y_pred)

    elif best_model_type == 'piecewise':
        results['type'] = 'piecewise'
        results['bath_slope1'] = slope1
        results['bath_slope2'] = slope2
        results['intercepts'] = (intercept1, intercept2, None)
        results['bp'] = best_bp
        y_pred = slope2 * X + intercept2
        y_pred[X >= best_bp] = slope1 * X[X >= best_bp] + intercept1
        results['r2'] = r2_score(y, y_pred)

    else:
        results['type'] = 'simple'
        results['bath_slope1'] = slope_simple
        results['intercepts'] = (intercept_simple, None, None)
        results['r2'] = r2_score(y, model_s.predict(X.reshape(-1, 1)))

    return results

def groupwise_regression(csv_path, min_width_range=40):
    """
    min_width_range: minimum span in width (m) each segment must have
    """
    df = pd.read_csv(csv_path)
    result_rows = []
    group_results = {}
    for node_id, group in df.groupby('node_id'):
        group = group.dropna(subset=['width', 'wse'])
        if len(group) < 3:
            continue
        X = group['width'].values
        y = group['wse'].values
        res = fit_regressions_for_group(X, y, min_width_range=min_width_range)
        result_rows.append({
            'node_id': node_id,
            'bath_slope1': res['bath_slope1'],
            'bath_slope2': res['bath_slope2'],
            'r2': res['r2'],
            'type': res['type']
        })
        group_results[node_id] = (group, res)
    result_df = pd.DataFrame(result_rows)
    return result_df, group_results

def plot_random_regressions(group_results, n=8):
    plot_ids = random.sample(list(group_results.keys()), min(n, len(group_results)))
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))
    axs = axs.flatten()
    for i, node_id in enumerate(plot_ids):
        group, res = group_results[node_id]
        X = group['width'].values
        y = group['wse'].values
        ax = axs[i]
        ax.scatter(X, y, label="Data", color='steelblue')
        x_plot = np.linspace(X.min(), X.max(), 100)

        if res['type'] == 'simple':
            slope, intercept = res['bath_slope1'], res['intercepts'][0]
            y_fit = slope * x_plot + intercept
            ax.plot(x_plot, y_fit, color='crimson', linewidth=2, label=f"Simple: slope={slope:.2f}")
        elif res['type'] == 'piecewise':
            slope1, intercept1 = res['bath_slope1'], res['intercepts'][0]
            slope2, intercept2 = res['bath_slope2'], res['intercepts'][1]
            bp = res['bp']
            y_left = slope2 * x_plot + intercept2
            y_right = slope1 * x_plot + intercept1
            ax.plot(x_plot[x_plot < bp], y_left[x_plot < bp], color='crimson', linewidth=2, label=f"Left: slope={slope2:.2f}")
            ax.plot(x_plot[x_plot >= bp], y_right[x_plot >= bp], color='crimson', linewidth=2, linestyle='--', label=f"Right: slope={slope1:.2f}")
            ax.axvline(bp, color='gray', linestyle=':', label=f"Breakpoint = {bp:.2f}")
        else:  # 'piecewise_2bp'
            slope1, slope2, slope3 = res['bath_slope1'], res['bath_slope2'], res['bath_slope3']
            intercept1, intercept2, intercept3 = res['intercepts']
            bp1, bp2 = res['bp']
            y_fit_left = slope3 * x_plot + intercept3
            y_fit_mid = slope2 * x_plot + intercept2
            y_fit_right = slope1 * x_plot + intercept1
            ax.plot(x_plot[x_plot < bp1], y_fit_left[x_plot < bp1], color='crimson', linewidth=2, label=f"Left: {slope3:.2f}")
            ax.plot(x_plot[(x_plot >= bp1) & (x_plot < bp2)], y_fit_mid[(x_plot >= bp1) & (x_plot < bp2)], color='crimson', linewidth=2, linestyle='--', label=f"Mid: {slope2:.2f}")
            ax.plot(x_plot[x_plot >= bp2], y_fit_right[x_plot >= bp2], color='crimson', linewidth=2, linestyle=':', label=f"Right: {slope1:.2f}")
            ax.axvline(bp1, color='gray', linestyle=':', label=f"BP1 = {bp1:.2f}")
            ax.axvline(bp2, color='gray', linestyle=':', label=f"BP2 = {bp2:.2f}")

        ax.set_title(f"node_id={node_id}\nR2={res['r2']:.2f}")
        ax.set_xlabel("Width")
        ax.set_ylabel("WSE")
        ax.legend()
        ax.grid(True)

    plt.tight_layout()
    plt.show()


path2w_wse = os.path.join(os.getcwd(), "2_intermediate/Store/Binary_Masks/Pee_dee/Bath_widths.csv")

In [ ]:
# Run the groupwise regression
result_df, group_results = groupwise_regression(path2w_wse, min_width_range=40)

# Show the DataFrame with slopes and r2
result_df

# Optionally, save to CSV
result_df.to_csv(os.path.join(os.getcwd(),"2_intermediate/Store/Binary_Masks/Pee_dee/Bath_slopes3.csv"), index=False)

# Plot 8 random node_id fits
plot_random_regressions(group_results, n=8)

### Regression code for dynamic threshold:

In [ ]:
def compute_aic(rss, n, k):
    return n * np.log(rss / n) + 2 * k

def fit_huber(X_input, y_input):
    model = HuberRegressor(epsilon=1.345, max_iter=500)
    model.fit(X_input, y_input)
    y_pred = model.predict(X_input)
    residuals = y_input - y_pred
    return model, y_pred, residuals

def fit_regressions_for_group(
    X, y,
    min_width_range=None,   # if None: dynamic span per group = alpha * {median|mean}(width)
    alpha=0.30,             # fraction for dynamic span (default 30%)
    basis='median'          # 'median' (default) or 'mean'
):
    """
    Fits Huber regressions (simple / 1-BP / 2-BP).
    A breakpoint is accepted only if EACH resulting segment's width span (np.ptp)
    is >= threshold. Threshold is:
        - min_width_range (absolute, if provided), otherwise
        - alpha * {median|mean}(width) computed per group (default: 30% of median).

    NOTE: Function name, keys, and overall behavior remain the same as your original.
    """
    n = len(X)
    results = {
        'best_model_type': 'simple',
        'bath_slope1': None,
        'bath_slope2': None,
        'bath_slope3': None,
        'r2': None,
        'intercepts': (None, None, None),
        'bp': None
    }

    # --- dynamic per-group span (default) ---
    if min_width_range is None:
        if basis == 'mean':
            min_width_range = alpha * float(np.nanmean(X))
        else:
            min_width_range = alpha * float(np.nanmedian(X))

    def is_invalid_slope(s):
        # invalid if None/NaN/inf or < 0.001 or negative
        return (s is None) or np.isnan(s) or np.isinf(s) or (s < 0.001)

    # === Simple model ===
    model_s, y_pred_s, res_s = fit_huber(X.reshape(-1, 1), y)
    rss_s = np.sum(res_s ** 2)
    aic_s = compute_aic(rss_s, n, 2)
    slope_simple = model_s.coef_[0]
    intercept_simple = model_s.intercept_
    best_model_type = 'simple'
    best_aic = aic_s

    slope1 = slope2 = slope3 = None
    intercept1 = intercept2 = intercept3 = None
    best_bp = None
    best_bps = None

    unique_x = np.unique(X)

    # === Try 2 breakpoints (three segments) ===
    if len(unique_x) >= 4:
        for i in range(1, len(unique_x) - 2):
            for j in range(i + 1, len(unique_x) - 1):
                bp1 = unique_x[i]
                bp2 = unique_x[j]
                if bp1 >= bp2:
                    continue

                seg1 = X[X < bp1]
                seg2 = X[(X >= bp1) & (X < bp2)]
                seg3 = X[X >= bp2]

                # need at least 2 points per side and sufficient width span (dynamic per group)
                if any(seg.size < 2 for seg in (seg1, seg2, seg3)):
                    continue
                if any(np.ptp(seg) < min_width_range for seg in (seg1, seg2, seg3)):
                    continue

                X_pw2 = np.column_stack([X, np.maximum(0, X - bp1), np.maximum(0, X - bp2)])
                try:
                    model_pw2, y_pred_pw2, res_pw2 = fit_huber(X_pw2, y)
                except Exception:
                    continue

                h0 = model_pw2.intercept_
                h1, d1, d2 = model_pw2.coef_

                slope_left  = h1
                slope_mid   = h1 + d1
                slope_right = h1 + d1 + d2

                # reject invalid / negative / too-flat segments
                if any(is_invalid_slope(s) for s in (slope_left, slope_mid, slope_right)):
                    continue

                intercept_left  = h0
                intercept_mid   = h0 - d1 * bp1
                intercept_right = h0 - d1 * bp1 - d2 * bp2

                rss_pw2 = np.sum(res_pw2 ** 2)
                aic_pw2 = compute_aic(rss_pw2, n, 4)  # 4 params: h0, h1, d1, d2

                if aic_pw2 < best_aic:
                    best_model_type = 'piecewise_2bp'
                    best_aic = aic_pw2
                    # keep naming convention: slope1 = rightmost (higher widths)
                    slope1, slope2, slope3 = slope_right, slope_mid, slope_left
                    intercept1, intercept2, intercept3 = intercept_right, intercept_mid, intercept_left
                    best_bps = (bp1, bp2)

    # === Try 1 breakpoint (two segments) ===
    if best_model_type == 'simple' and len(unique_x) >= 3:
        for bp in unique_x[1:-1]:
            seg_left = X[X < bp]
            seg_right = X[X >= bp]

            if seg_left.size < 2 or seg_right.size < 2:
                continue
            if (np.ptp(seg_left) < min_width_range) or (np.ptp(seg_right) < min_width_range):
                continue

            X_pw1 = np.column_stack([X, np.maximum(0, X - bp)])
            try:
                model_pw1, y_pred_pw1, res_pw1 = fit_huber(X_pw1, y)
            except Exception:
                continue

            h0 = model_pw1.intercept_
            h1, d1 = model_pw1.coef_

            slope_left  = h1
            slope_right = h1 + d1

            if any(is_invalid_slope(s) for s in (slope_left, slope_right)):
                continue

            intercept_left  = h0
            intercept_right = h0 - d1 * bp

            rss_pw1 = np.sum(res_pw1 ** 2)
            aic_pw1 = compute_aic(rss_pw1, n, 3)  # 3 params: h0, h1, d1

            if aic_pw1 < best_aic:
                best_model_type = 'piecewise'
                best_aic = aic_pw1
                # naming convention: slope1 = right (higher widths), slope2 = left
                slope1, slope2 = slope_right, slope_left
                intercept1, intercept2 = intercept_right, intercept_left
                best_bp = bp

    # === Store results (match original keys) ===
    if best_model_type == 'piecewise_2bp':
        results['type'] = 'piecewise_2bp'
        results['bath_slope1'] = slope1
        results['bath_slope2'] = slope2
        results['bath_slope3'] = slope3
        results['intercepts'] = (intercept1, intercept2, intercept3)
        results['bp'] = best_bps
        y_pred = np.empty_like(y)
        y_pred[:] = slope3 * X + intercept3
        y_pred[X >= best_bps[0]] = slope2 * X[X >= best_bps[0]] + intercept2
        y_pred[X >= best_bps[1]] = slope1 * X[X >= best_bps[1]] + intercept1
        results['r2'] = r2_score(y, y_pred)

    elif best_model_type == 'piecewise':
        results['type'] = 'piecewise'
        results['bath_slope1'] = slope1
        results['bath_slope2'] = slope2
        results['intercepts'] = (intercept1, intercept2, None)
        results['bp'] = best_bp
        y_pred = slope2 * X + intercept2
        y_pred[X >= best_bp] = slope1 * X[X >= best_bp] + intercept1
        results['r2'] = r2_score(y, y_pred)

    else:
        results['type'] = 'simple'
        results['bath_slope1'] = slope_simple
        results['intercepts'] = (intercept_simple, None, None)
        results['r2'] = r2_score(y, model_s.predict(X.reshape(-1, 1)))

    return results

def groupwise_regression(
    csv_path,
    min_width_range=None,  # if provided: fixed absolute span (meters). If None: dynamic span.
    alpha=0.30,            # fraction for dynamic span (default 30%)
    basis='median',        # 'median' (default) or 'mean'
    global_stat=False      # if True: use one global statistic for all groups
):
    """
    min_width_range:
        - If provided (number), use a fixed minimum width span (meters) for all groups (original behavior).
        - If None (default), use dynamic span = alpha * {median|mean}(width):
            * per group (default), OR
            * global across the whole dataset (if global_stat=True).

    Prints, for transparency:
      - Global median/mean (if global_stat=True)
      - Per-group median/mean and the min_width_span_used
    """
    df = pd.read_csv(csv_path)
    result_rows = []
    group_results = {}

    # --- compute global statistic if needed ---
    if global_stat:
        if basis == 'mean':
            global_val = float(np.nanmean(df['width']))
        else:
            global_val = float(np.nanmedian(df['width']))
        print(f"Global {basis} width = {global_val:.2f}")
    else:
        global_val = None

    for node_id, group in df.groupby('node_id'):
        group = group.dropna(subset=['width', 'wse'])
        if len(group) < 3:
            continue
        X = group['width'].values
        y = group['wse'].values

        # compute width statistic used for this group
        if global_stat:
            group_stat = global_val
        else:
            if basis == 'mean':
                group_stat = float(np.nanmean(X))
            else:
                group_stat = float(np.nanmedian(X))

        # decide dynamic or fixed span
        if min_width_range is None:
            dyn_span = alpha * group_stat
        else:
            dyn_span = float(min_width_range)

        # print per-group info
        print(f"node_id={node_id}: {basis} width = {group_stat:.2f}, min_width_span_used = {dyn_span:.2f}")

        res = fit_regressions_for_group(
            X, y,
            min_width_range=dyn_span,
            alpha=alpha,
            basis=basis
        )
        result_rows.append({
            'node_id': node_id,
            'bath_slope1': res['bath_slope1'],
            'bath_slope2': res['bath_slope2'],
            'r2': res['r2'],
            'type': res['type'],
            'width_stat': group_stat,
            'min_width_span_used': dyn_span
        })
        group_results[node_id] = (group, res)

    result_df = pd.DataFrame(result_rows)
    return result_df, group_results

def plot_random_regressions(group_results, n=8):
    plot_ids = random.sample(list(group_results.keys()), min(n, len(group_results)))
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))
    axs = axs.flatten()
    for i, node_id in enumerate(plot_ids):
        group, res = group_results[node_id]
        X = group['width'].values
        y = group['wse'].values
        ax = axs[i]
        ax.scatter(X, y, label="Data", color='steelblue')
        x_plot = np.linspace(X.min(), X.max(), 100)

        if res['type'] == 'simple':
            slope, intercept = res['bath_slope1'], res['intercepts'][0]
            y_fit = slope * x_plot + intercept
            ax.plot(x_plot, y_fit, color='crimson', linewidth=2, label=f"Simple: slope={slope:.2f}")
        elif res['type'] == 'piecewise':
            slope1, intercept1 = res['bath_slope1'], res['intercepts'][0]
            slope2, intercept2 = res['bath_slope2'], res['intercepts'][1]
            bp = res['bp']
            y_left = slope2 * x_plot + intercept2
            y_right = slope1 * x_plot + intercept1
            ax.plot(x_plot[x_plot < bp], y_left[x_plot < bp], color='crimson', linewidth=2, label=f"Left: slope={slope2:.2f}")
            ax.plot(x_plot[x_plot >= bp], y_right[x_plot >= bp], color='crimson', linewidth=2, linestyle='--', label=f"Right: slope={slope1:.2f}")
            ax.axvline(bp, color='gray', linestyle=':', label=f"Breakpoint = {bp:.2f}")
        else:  # 'piecewise_2bp'
            slope1, slope2, slope3 = res['bath_slope1'], res['bath_slope2'], res['bath_slope3']
            intercept1, intercept2, intercept3 = res['intercepts']
            bp1, bp2 = res['bp']
            y_fit_left = slope3 * x_plot + intercept3
            y_fit_mid = slope2 * x_plot + intercept2
            y_fit_right = slope1 * x_plot + intercept1
            ax.plot(x_plot[x_plot < bp1], y_fit_left[x_plot < bp1], color='crimson', linewidth=2, label=f"Left: {slope3:.2f}")
            ax.plot(x_plot[(x_plot >= bp1) & (x_plot < bp2)], y_fit_mid[(x_plot >= bp1) & (x_plot < bp2)], color='crimson', linewidth=2, linestyle='--', label=f"Mid: {slope2:.2f}")
            ax.plot(x_plot[x_plot >= bp2], y_fit_right[x_plot >= bp2], color='crimson', linewidth=2, linestyle=':', label=f"Right: {slope1:.2f}")
            ax.axvline(bp1, color='gray', linestyle=':', label=f"BP1 = {bp1:.2f}")
            ax.axvline(bp2, color='gray', linestyle=':', label=f"BP2 = {bp2:.2f}")

        ax.set_title(f"node_id={node_id}\nR2={res['r2']:.2f}")
        ax.set_xlabel("Width")
        ax.set_ylabel("WSE")
        ax.legend()
        ax.grid(True)

    plt.tight_layout()
    plt.show()


path2w_wse = os.path.join(os.getcwd(), "2_intermediate/Store/Binary_Masks/Po/Bath_widths.csv")

In [ ]:
result_df, group_results = groupwise_regression(path2w_wse, global_stat=False, alpha=0.07, basis='mean')
# Optionally, save to CSV
result_df.to_csv(os.path.join(os.getcwd(),"2_intermediate/Store/Binary_Masks/Garonne/Bath_slopes3.csv"), index=False)
# Plot 8 random node_id fits
plot_random_regressions(group_results, n=8)


In [ ]:
def plot_all_regressions(group_results):
    import math

    n = len(group_results)
    if n == 0:
        print("No groups to plot.")
        return

    ncols = 4
    nrows = math.ceil(n / ncols)

    fig, axs = plt.subplots(nrows, ncols, figsize=(5 * ncols, 4 * nrows))
    # Ensure axs is always a flat iterable
    if isinstance(axs, np.ndarray):
        axs = axs.flatten()
    else:
        axs = [axs]

    for i, (node_id, (group, res)) in enumerate(group_results.items()):
        X = group['width'].values
        y = group['wse'].values
        ax = axs[i]
        ax.scatter(X, y, label="Data", color='steelblue')

        x_plot = np.linspace(X.min(), X.max(), 300)
        model_type = res['type']

        if model_type == 'simple':
            slope, intercept = res['bath_slope1'], res['intercepts'][0]
            y_fit = slope * x_plot + intercept
            ax.plot(x_plot, y_fit, color='crimson', linewidth=2, label=f"Simple: slope={slope:.4f}")

        elif model_type == 'piecewise':
            slope1, intercept1 = res['bath_slope1'], res['intercepts'][0]
            slope2, intercept2 = res['bath_slope2'], res['intercepts'][1]
            bp = res['bp']
            y_left = slope2 * x_plot + intercept2
            y_right = slope1 * x_plot + intercept1
            ax.plot(x_plot[x_plot < bp], y_left[x_plot < bp], color='crimson', label=f"Left: slope={slope2:.4f}")
            ax.plot(x_plot[x_plot >= bp], y_right[x_plot >= bp], color='crimson', linestyle='--', label=f"Right: slope={slope1:.4f}")
            ax.axvline(bp, color='gray', linestyle=':', label=f"Breakpoint = {bp:.2f}")

        elif model_type == 'piecewise_2bp':
            slope1, slope2, slope3 = res['bath_slope1'], res['bath_slope2'], res['bath_slope3']
            intercept1, intercept2, intercept3 = res['intercepts']
            bp1, bp2 = res['bp']

            y_fit_left = slope3 * x_plot + intercept3
            y_fit_mid = slope2 * x_plot + intercept2
            y_fit_right = slope1 * x_plot + intercept1

            ax.plot(x_plot[x_plot < bp1], y_fit_left[x_plot < bp1], color='crimson', label=f"Left: {slope3:.4f}")
            ax.plot(x_plot[(x_plot >= bp1) & (x_plot < bp2)], y_fit_mid[(x_plot >= bp1) & (x_plot < bp2)], color='crimson', linestyle='--', label=f"Mid: {slope2:.4f}")
            ax.plot(x_plot[x_plot >= bp2], y_fit_right[x_plot >= bp2], color='crimson', linestyle=':', label=f"Right: {slope1:.4f}")
            ax.axvline(bp1, color='gray', linestyle=':', label=f"BP1 = {bp1:.2f}")
            ax.axvline(bp2, color='gray', linestyle=':', label=f"BP2 = {bp2:.2f}")

        ax.set_title(f"node_id={node_id}\nR² = {res['r2']:.2f}")
        ax.set_xlabel("Width")
        ax.set_ylabel("WSE")
        ax.legend()
        ax.grid(True)

    # Remove any extra unused subplots
    total_axes = nrows * ncols
    for j in range(n, total_axes):
        fig.delaxes(axs[j])

    plt.tight_layout()
    plt.show()



In [ ]:
plot_all_regressions(group_results)

Merge csv file with Bathymetry slopes with the Regression master csv file

In [ ]:
def merge_csv(master_csv, complementary_csv, output_csv):
    """
    Merge two CSV files on 'node_id' and save the result.
    """
    master_df = pd.read_csv(master_csv)
    comp_df = pd.read_csv(complementary_csv)

    # Merge on 'node_id'
    merged_df = master_df.merge(comp_df, on='node_id', how='inner')

    # Save the merged DataFrame
    merged_df.to_csv(output_csv, index=False)
    print(f"Merged CSV saved to {output_csv}")
    
    return merged_df

path2slope = os.path.join(os.getcwd(), "2_intermediate/Store/Binary_Masks/Garonne/Bath_slopes3.csv")
outpath = os.path.join(os.getcwd(), "2_intermediate/Store/Validation/csv/Garonne/reg_slope_bath3.csv")

In [ ]:
reg_slope_bath = merge_csv(master_csv_path, path2slope, outpath)

In [ ]:
reg_slope_bath['node_id'].nunique()

In [ ]:
reg_slope_bath_sort = reg_slope_bath.sort_values('p_dist_out', ascending=False).groupby('node_id', as_index=False).first()

In [ ]:
## Outlier removal in SWOT and bathymetry
Q1, Q3 = reg_slope_bath_sort['slope1'].quantile([0.25, 0.75])
IQR = Q3 - Q1
l1, u1 = Q1 - 1.5*IQR, Q3 + 1.5*IQR

# bath_slope1 fences
Q1b, Q3b = reg_slope_bath_sort['bath_slope1'].quantile([0.25, 0.75])
IQRb   = Q3b - Q1b
l2, u2 = Q1b - 1.5*IQRb, Q3b + 1.5*IQRb

In [ ]:
mask = (
    (reg_slope_bath_sort['slope1']      >= l1) & (reg_slope_bath_sort['slope1']      <= u1) &
    (reg_slope_bath_sort['bath_slope1'] >= l2) & (reg_slope_bath_sort['bath_slope1'] <= u2)
)
reg_slope_bath_sort = reg_slope_bath_sort[mask]

In [ ]:
reg_slope_bath_sort['node_id'].nunique()

In [ ]:
# --- 0) Work on a copy so you can compare before/after if needed
df = reg_slope_bath_sort.copy()

# --- 1) Compute rel error (percent) in-place-friendly columns
# (safe divide: ignore zero/NaN denominators in the trimming stats)
rel = 100 * (df['slope1'] - df['bath_slope1']) / df['bath_slope1']
df['rel_error'] = rel
df['abs_rel_error'] = rel.abs()

# --- 2) Build a UNIQUE view (like your CDF step) for threshold estimation only
uniq = df[['slope1','bath_slope1','rel_error']].dropna().drop_duplicates(subset=['slope1','bath_slope1'])

# --- 3) MAD right-tail rule (robust, no small-slope dropping)
med  = uniq['rel_error'].median()
mad0 = (uniq['rel_error'] - med).abs().median()
mad  = 1.4826 * mad0                # #1.4826 makes MAD behave like std deviation
k    = 3.0                          # tweakable: 2.5–4. tweak: bigger = keep more, smaller = remove more. it’s analogous to saying “flag anything more than 3σ above the median.”
thr_mad = med + k * mad

mask_out_mad = uniq['rel_error'] > thr_mad

# --- 4) Optional percentile tail shave (set q=None to skip)
q = 0.99                            # top 1% cutoff; set to None to disable
if q is not None:
    thr_q = uniq['rel_error'].quantile(q)
    mask_out_pct = uniq['rel_error'] > thr_q
    mask_out_all = mask_out_mad | mask_out_pct
else:
    mask_out_all = mask_out_mad

# --- 5) Map the unique outliers back to the full df (keep structure)
out_pairs = uniq.loc[mask_out_all, ['slope1','bath_slope1']]
out_pairs['__key__'] = 1
df_key = df[['slope1','bath_slope1']].copy()
df_key['__key__'] = 1

# mark matches
to_drop = df_key.merge(out_pairs, on=['__key__','slope1','bath_slope1'], how='left', indicator=True)
is_rel_outlier = (to_drop['_merge'] == 'both').values

# (optional) keep a flag and a log before dropping
df['is_rel_outlier'] = is_rel_outlier
removed = df[df['is_rel_outlier']].copy()
kept    = df[~df['is_rel_outlier']].copy()

# --- 6) Replace reg_slope_bath_sort with the cleaned version (same columns as before)
# If you want the exact original schema, drop the helper columns:
reg_slope_bath_sort = kept.drop(columns=['rel_error','abs_rel_error','is_rel_outlier'])

In [ ]:
reg_slope_bath_sort['node_id'].nunique()

Error metrics and coefficient of correlations

In [ ]:
x = reg_slope_bath_sort['slope1'].values
y = reg_slope_bath_sort['bath_slope1'].values

# Classic metrics
pearson_corr, pearson_p = pearsonr(x, y)
spearman_corr, spearman_p = spearmanr(x, y)
mae = mean_absolute_error(y, x)
rmse = np.sqrt(mean_squared_error(y, x))

# Relative errors
rel_error = 100 * (x - y) / y
mean_bias = np.mean(rel_error)
mean_abs_rel_error = np.mean(np.abs(rel_error))
percentile_68 = np.percentile(np.abs(rel_error), 68)

print(f"Pearson correlation: {pearson_corr:.3f} (p={pearson_p:.2g})")
print(f"Spearman correlation: {spearman_corr:.3f} (p={spearman_p:.2g})")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Relative Error (Bias): {mean_bias:.2f}%")
print(f"Mean Absolute Relative Error: {mean_abs_rel_error:.2f}%")
print(f"68th percentile Absolute Relative Error: {percentile_68:.2f}%")

By reach

In [ ]:
def segment_by_spearman(
    df: pd.DataFrame,
    rho_threshold: float = 0.4,
    pval_threshold: float = 0.05,
    min_length: int = 10,
    method: str = 'spearman'   # 'spearman' or 'pearson' for thresholding
):
    if method not in ('spearman','pearson'):
        raise ValueError("method must be 'spearman' or 'pearson'")
    corr_test = spearmanr if method == 'spearman' else pearsonr

    # Keep original index
    df = df.copy()
    orig_index = df.index.to_numpy()

    n = len(df)
    in_segment = np.zeros(n, dtype=bool)
    seg_ids    = np.full(n, np.nan)
    segments   = []
    flagged_pts = []

    i = 0
    next_seg_id = 1
    while i < n:
        best_end = None

        # Slide j to find the longest valid block
        for j in range(i + min_length, n + 1):
            block = df.iloc[i:j]
            corr_val, pval = corr_test(block['slope1'], block['bath_slope1'])
            if (corr_val >= rho_threshold) and (pval <= pval_threshold):
                best_end = j

        if best_end:
            seg = df.iloc[i:best_end]
            x, y = seg['slope1'].values, seg['bath_slope1'].values

            # Report both metrics
            pr, prp = pearsonr(x, y)
            sr, srp = spearmanr(x, y)
            mae     = mean_absolute_error(y, x)
            rmse    = np.sqrt(mean_squared_error(y, x))
            r2      = r2_score(y, x)

            rel_err = 100 * (x - y) / y
            mean_rel = float(np.mean(rel_err))
            mean_abs = float(np.mean(np.abs(rel_err)))
            pct68    = float(np.percentile(np.abs(rel_err), 68))

            segments.append({
                'segment_id':             next_seg_id,
                'start_dist':             seg['p_dist_out'].iloc[0],
                'end_dist':               seg['p_dist_out'].iloc[-1],
                'n_points':               len(seg),
                'pearson':                pr,
                'pearson_p':              prp,
                'spearman':               sr,
                'spearman_p':             srp,
                'mae':                    mae,
                'rmse':                   rmse,
                'r2':                     r2,
                'mean_rel_error':         mean_rel,
                'mean_abs_rel_error':     mean_abs,
                'pctile_68_abs_rel_err':  pct68
            })

            # Mark membership
            in_segment[i:best_end] = True
            seg_ids[i:best_end]    = next_seg_id
            next_seg_id += 1
            i = best_end
        else:
            row = df.iloc[i].to_dict()
            row['orig_index'] = int(orig_index[i])
            row['flag_reason'] = 'no_valid_block'
            flagged_pts.append(row)
            i += 1

    segments_df = pd.DataFrame(segments)
    flagged_df  = pd.DataFrame(flagged_pts)

    # Build the full annotated dataframe
    annotated_df = df.copy()
    annotated_df['segment_id'] = seg_ids
    annotated_df['flagged'] = in_segment  # True = in a valid segment

    # === PRINT REPORT AS BEFORE ===
    print("=== Segments report ===")
    print(segments_df)
    print("\n=== Flagged points (couldn't join any valid block) ===")
    print(flagged_df)
    print(f"\nTotal rows: {len(df)}, In segment: {in_segment.sum()}, Flagged: {(~in_segment).sum()}")

    return segments_df, flagged_df, annotated_df

In [ ]:
segments_df, flagged_df, annotated_df = segment_by_spearman(
    reg_slope_bath_sort,
    rho_threshold=0.4,
    pval_threshold=0.05,
    min_length=10,
    method='spearman'
)

In [ ]:
# === Export flagged_df with original column order ===
cols = list(reg_slope_bath_sort.columns)   # original order
extra = [c for c in ['segment_id','flagged'] if c not in cols]
annotated_df[cols + extra].to_csv(os.path.join(os.getcwd(), "2_intermediate/Store/Validation/csv/Pee_dee/flagged_nodes_80.csv"), index=False)

Figures for all nodes, including bad spearmans

In [ ]:
rel_error = 100 * (reg_slope_bath_sort['slope1'] - reg_slope_bath_sort['bath_slope1']) / reg_slope_bath_sort['bath_slope1']
abs_rel_error = np.abs(rel_error)

# Drop duplicates for (slope1, bath_slope1) pairs (optional but matches your logic)
df_rel = pd.DataFrame({
    'slope1': reg_slope_bath_sort['slope1'],
    'bath_slope1': reg_slope_bath_sort['bath_slope1'],
    'rel_error': rel_error,
    'abs_rel_error': abs_rel_error
})
df_rel_unique = df_rel.drop_duplicates(subset=['slope1', 'bath_slope1'], keep='first')

# Sorted arrays for CDFs
bias_sorted = np.sort(df_rel_unique['rel_error'].values)
cdf_bias = np.arange(1, len(bias_sorted) + 1) / len(bias_sorted)

abs_sorted = np.sort(df_rel_unique['abs_rel_error'].values)
cdf_abs = np.arange(1, len(abs_sorted) + 1) / len(abs_sorted)

# Metrics
mean_bias = df_rel_unique['rel_error'].mean()
mean_abs_rel_error = df_rel_unique['abs_rel_error'].mean()
spearman_rho, spearman_p = spearmanr(df_rel_unique['slope1'], df_rel_unique['bath_slope1'])
p68 = np.percentile(df_rel_unique['abs_rel_error'], 68)

# Colors (customize as desired)
color_bias = '#ff5722'
color_abs = '#fbc02d'
color_p68 = 'dimgrey'
color_zero = '#ffa726'
color_mean = '#2979ff'

# ------------------------------------------------------------------------------
# --- Plotting ---
# ------------------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_facecolor('#f5f5f5')

# Plot Relative Error CDF
ax.scatter(
    bias_sorted, cdf_bias,
    color=color_bias,
    label='Relative Error CDF (bias, %)',
    s=70,
    edgecolors='black',
    linewidth=0.5,
    marker='o',
    alpha=0.7
)

# Plot Absolute Relative Error CDF
ax.scatter(
    abs_sorted, cdf_abs,
    color=color_abs,
    label='Absolute Relative Error CDF (%)',
    s=25,
    edgecolors='black',
    linewidth=0.5,
    marker='D',
    alpha=0.9
)

# Zero-Bias vertical line
ax.axvline(
    x=0,
    color=color_zero,
    linestyle='--',
    linewidth=1.8,
    label='Zero Bias'
)

# Mean-Bias vertical line
ax.axvline(
    x=mean_bias,
    color=color_mean,
    linestyle='-.',
    linewidth=2,
    label=f'Mean Bias = {mean_bias:.2f} %'
)

# 68th-percentile vertical/horizontal lines
ax.axvline(
    x=p68,
    color=color_p68,
    linestyle=':',
    linewidth=2.5,
    label=f'|68%ile|: {p68:.2f} %'
)
ax.axhline(
    y=0.68,
    color=color_p68,
    linestyle=':',
    linewidth=2.5
)

# Axis limits and ticks
xmin = np.floor(min(bias_sorted.min(), abs_sorted.min())) - 1
xmax = np.ceil(max(bias_sorted.max(), abs_sorted.max(), p68)) + 1
ax.set_xlim(xmin, xmax)
ax.set_xticks(np.arange(xmin, xmax + 1, 20))

# Labels and title
ax.set_xlabel('Relative Error (%)', fontsize=14)
ax.set_ylabel('Cumulative Probability', fontsize=14)
ax.set_title(
    'CDF of Slope Relative Error, Mean Bias and 68%tile\n',
    fontsize=16,
    weight='bold'
)

# Grid and legend
ax.grid(True, which='major', linestyle='--', alpha=0.5)
ax.legend(fontsize=12, loc='lower right', frameon=True, fancybox=True)

# ------------------------------------------------------------------------------
# --- Marginal KDE plot for bias (inset at top) ---
# ------------------------------------------------------------------------------
ax_kde = inset_axes(
    ax,
    width="100%", height="20%",
    loc='upper center',
    bbox_to_anchor=(0, 1.05, 1, 0.3),
    bbox_transform=ax.transAxes,
    borderpad=0
)
sns.kdeplot(
    x=df_rel_unique['rel_error'],
    ax=ax_kde,
    fill=True,
    linewidth=2,
    alpha=0.3
)
ax_kde.set_xlim(xmin, xmax)
ax_kde.axis('off')

plt.tight_layout()
plt.show()

# --- Print metrics summary ---
print(f"Spearman correlation: {spearman_rho:.3f} (p={spearman_p:.2g})")
print(f"Mean Bias (rel error): {mean_bias:.2f} %")
print(f"Mean Absolute Relative Error: {mean_abs_rel_error:.2f} %")
print(f"68th percentile abs rel error: {p68:.2f} %")



In [ ]:
s1 = reg_slope_bath_sort['slope1']
s2 = reg_slope_bath_sort['bath_slope1']
x  = reg_slope_bath_sort['p_dist_out']

# ─── figure setup ───────────────────────────────────────────────
fig = plt.figure(figsize=(15,6), facecolor='white')
gs  = GridSpec(1, 2, width_ratios=[5, 1.5], wspace=0.15)

# vibrant palette
c1, c2 = '#1f77b4', '#ff7f0e'

# ─── left: line vs. distance with thinner lines ────────────────
ax_main = fig.add_subplot(gs[0])

ax_main.plot(
    x, s1,
    color=c1,
    linestyle='-',
    linewidth=1.5,          # ← thinner line
    marker='o',
    markersize=5,           # ← small markers
    markerfacecolor='white',
    markeredgecolor=c1,
    markeredgewidth=1.0,
    label='SWOT slope'
)
ax_main.plot(
    x, s2,
    color=c2,
    linestyle='--',
    linewidth=1.5,          # ← thinner line
    marker='D',
    markersize=5,           # ← small markers
    markerfacecolor='white',
    markeredgecolor=c2,
    markeredgewidth=1.0,
    label='Bathymetry slope'
)

# gentle fill under curves
ax_main.fill_between(x, s1, alpha=0.1, color=c1)
ax_main.fill_between(x, s2, alpha=0.1, color=c2)

ax_main.set_xlabel('Distance to outlet (m)', fontsize=12, weight='bold')
ax_main.set_ylabel('Slope value', fontsize=12, weight='bold')
ax_main.set_title('Comparison between SWOT and Bathymetry Slopes', fontsize=16, weight='bold')
ax_main.grid(alpha=0.3, linestyle='--')
ax_main.invert_xaxis()
ax_main.legend(loc='upper left', frameon=True, facecolor='white', edgecolor='black')

# ─── right: side boxplot sharing the same y-axis ───────────────
ax_box = fig.add_subplot(gs[1], sharey=ax_main)

bp = ax_box.boxplot(
    [s1, s2],
    positions=[1,2],
    widths=0.6,
    patch_artist=True,
    showfliers=False,
    medianprops=dict(color='black'),
    whiskerprops=dict(color='black'),
    capprops=dict(color='black'),
)

# color the boxes with semi-transparent fill
for patch, color in zip(bp['boxes'], [c1, c2]):
    patch.set_facecolor(color)
    patch.set_alpha(0.3)

# overlay the raw points on top
for i, (y, color) in enumerate(zip([s1, s2], [c1, c2]), start=1):
    xi = np.random.normal(i, 0.04, size=len(y))
    ax_box.scatter(
        xi, y,
        color=color,
        alpha=0.6,
        s=12,
        edgecolors='none',
        zorder=10
    )

ax_box.set_xticks([1,2])
ax_box.set_xticklabels(['SWOT', 'Bathymetry'], fontsize=10, weight='bold')
ax_box.margins(x=0.3)
ax_box.yaxis.tick_right()
ax_box.yaxis.set_label_position("right")
ax_box.grid(True, axis='y', linestyle='--', alpha=0.4)

plt.tight_layout()
plt.show()


In [ ]:
s1 = reg_slope_bath_sort['slope1']
s2 = reg_slope_bath_sort['bath_slope1']

# set up the figure
plt.figure(figsize=(6,6))
plt.scatter(s1, s2, 
            color='red',      # match your SWOT color
            alpha=0.7, 
            edgecolors='none', 
            s=30)

# 1:1 line
lims = [
    np.min([s1.min(), s2.min()]), 
    np.max([s1.max(), s2.max()])
]
plt.plot(lims, lims, 
        ls='--', 
        lw=1.5, 
        color='green', 
        label='1:1 line')

margin = 0.001
low, high = lims

plt.xlim(low - margin, high + margin)
plt.ylim(low - margin, high + margin)

plt.xlabel('SWOT slope', fontsize=12, weight='bold')
plt.ylabel('Bathymetry slope', fontsize=12, weight='bold')
plt.title('SWOT vs. Bathymetry Slopes', fontsize=14, weight='bold')
plt.grid(alpha=0.3, linestyle='--')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
s1 = reg_slope_bath_sort['slope1']           # SWOT slope
s2 = reg_slope_bath_sort['bath_slope1']      # Bathymetry slope

# 1) Hexbin + 1:1 line
plt.figure(figsize=(6,6))
plt.hexbin(s1, s2, gridsize=100, mincnt=1, cmap='Blues')  # no color specified → default cmap
lims = [min(s1.min(), s2.min()), max(s1.max(), s2.max())]
plt.plot(lims, lims, ls='--', lw=1.5, color='green', label='1:1 line')
plt.xlim(lims)
plt.ylim(lims)
plt.xlabel('SWOT slope', fontsize=12, weight='bold')
plt.ylabel('Bathymetry slope', fontsize=12, weight='bold')
plt.title('Hexbin: SWOT vs Bath Slopes', fontsize=14, weight='bold')
plt.legend()
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()


# 2) Bland–Altman (Difference vs Average)
diff = s1 - s2
avg  = 0.5 * (s1 + s2)
mean_diff = diff.mean()
std_diff  = diff.std()

plt.figure(figsize=(6,4))
plt.scatter(avg, diff, s=20, alpha=0.5)
plt.axhline(mean_diff, color='red', linestyle='--', label='Mean diff')
plt.axhline(mean_diff + 1.96*std_diff, color='gray', linestyle=':', label='±1.96σ')
plt.axhline(mean_diff - 1.96*std_diff, color='gray', linestyle=':')
plt.xlabel('Average slope', fontsize=12, weight='bold')
plt.ylabel('Difference (SWOT – Bath)', fontsize=12, weight='bold')
plt.title('Bland–Altman: Difference vs. Average', fontsize=14, weight='bold')
plt.legend()
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()


# 3) Residuals vs SWOT slope
plt.figure(figsize=(6,4))
plt.scatter(s1, diff, s=20, alpha=0.5)
plt.axhline(0, color='black', linestyle='--', lw=1)
plt.xlabel('SWOT slope', fontsize=12, weight='bold')
plt.ylabel('Residual (SWOT – Bath)', fontsize=12, weight='bold')
plt.title('Residuals vs. SWOT Slope', fontsize=14, weight='bold')
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()


plt.show()

# 4) CDF of Absolute Residuals (|SWOT – Bath|)
abs_diff = np.abs(diff)
abs_diff = abs_diff[np.isfinite(abs_diff)]
xs = np.sort(abs_diff)
cdf = np.arange(1, len(xs) + 1) / len(xs)

p50 = np.percentile(abs_diff, 50)
p68 = np.percentile(abs_diff, 68)
p95 = np.percentile(abs_diff, 95)

plt.figure(figsize=(6,4))
plt.plot(xs, cdf, lw=2)
plt.axvline(p68, linestyle='--', label=f'68%tile = {p68:.4f}')
plt.axvline(p50, linestyle=':',  label=f'Median = {p50:.4f}')
plt.axvline(p95, linestyle=':',  label=f'95%tile = {p95:.4f}')
plt.xlabel('|Residual| (|SWOT – Bath|)', fontsize=12, weight='bold')
plt.ylabel('Cumulative Probability', fontsize=12, weight='bold')
plt.title('CDF of Absolute Residuals', fontsize=14, weight='bold')
plt.legend()
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()
# (Alternative) Histogram of signed residuals
plt.figure(figsize=(6,4))
plt.hist(diff[np.isfinite(diff)], bins=30, density=True, alpha=0.7)
plt.axvline(mean_diff, linestyle='--', label=f'Mean = {mean_diff:.4f}')
plt.xlabel('Residual (SWOT – Bath)', fontsize=12, weight='bold')
plt.ylabel('Density', fontsize=12, weight='bold')
plt.title('Residual Distribution', fontsize=14, weight='bold')
plt.legend()
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()

# Histogram of Residuals
plt.figure(figsize=(6,4))
plt.hist(diff[np.isfinite(diff)], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(0, color='black', linestyle='--', lw=1, label='Zero line')
plt.axvline(mean_diff, color='red', linestyle='--', lw=1, label=f'Mean = {mean_diff:.4f}')
plt.xlabel('Residual (SWOT – Bath)', fontsize=12, weight='bold')
plt.ylabel('Count', fontsize=12, weight='bold')
plt.title('Histogram of Residuals', fontsize=14, weight='bold')
plt.legend()
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()

import seaborn as sns

# Clean residuals
diff_clean = np.asarray(diff[np.isfinite(diff)])

# Stats
q1, med, q3 = np.percentile(diff_clean, [25, 50, 75])
mean_val    = diff_clean.mean()
p68         = np.percentile(diff_clean, 68)

# Keep comparable limits
ymin = np.percentile(diff_clean, 1)
ymax = np.percentile(diff_clean, 99)
pad  = 0.02 * (ymax - ymin)
ylim = (ymin - pad, ymax + pad)

plt.figure(figsize=(5,4))

# Violin plot with same proportional look
ax = sns.violinplot(
    y=diff_clean,
    inner='box',
    color='lightblue',
    cut=0,
    scale='width',
    bw=0.2,
    linewidth=1.2
)

# Zero line
ax.axhline(0, color='black', linestyle='--', lw=1)

# Markers for stats
ax.scatter(0, mean_val, s=60, marker='D', color='#d62728', zorder=5, label=f"Mean = {mean_val:.4f}")
ax.scatter(0, med,      s=60, marker='o', facecolors='white', edgecolors='black', zorder=5, label=f"Median = {med:.4f}")
ax.scatter(0, q1,       s=80, marker='_', color='#2ca02c', zorder=5, label=f"Q1 = {q1:.4f}")
ax.scatter(0, q3,       s=80, marker='_', color='#1f77b4', zorder=5, label=f"Q3 = {q3:.4f}")
ax.axhline(p68, color='purple', linestyle=':', lw=1.2, label=f"68%tile = {p68:.4f}")

# Labels, limits, legend
ax.set_ylim(ylim)
ax.set_ylabel('Residual (SWOT - Bath)', fontsize=12, weight='bold')
ax.set_title('Residual Distribution Around Zero', fontsize=14, weight='bold')
ax.set_xticks([])

ax.legend(loc='lower left', fontsize=9, frameon=True)  # legend in lower-left

plt.tight_layout()
plt.show()



## By reach

In [ ]:
def _stats_report_for_group(g: pd.DataFrame):
    """Compute metrics for one consecutive TRUE group (one segment_id)."""
    # Clean copy
    g = g.copy()

    # Core vectors
    x_dist = g['p_dist_out'].values
    s1 = g['slope1'].values        # SWOT
    s2 = g['bath_slope1'].values   # Bath

    # Metrics (use non-NaNs only)
    m = np.isfinite(s1) & np.isfinite(s2)
    s1c, s2c = s1[m], s2[m]

    # If too few points, return minimal info
    if len(s1c) < 2:
        return {
            "pearson": np.nan, "pearson_p": np.nan,
            "spearman": np.nan, "spearman_p": np.nan,
            "mae": np.nan, "rmse": np.nan, "r2": np.nan,
            "mean_rel": np.nan, "mean_abs_rel": np.nan, "p68_abs_rel": np.nan
        }

    pr, prp = pearsonr(s1c, s2c)
    sr, srp = spearmanr(s1c, s2c)
    mae     = mean_absolute_error(s2c, s1c)
    rmse    = np.sqrt(mean_squared_error(s2c, s1c))
    r2      = r2_score(s2c, s1c)

    # Relative errors (drop dup pairs as in your logic)
    df_rel = pd.DataFrame({"slope1": s1c, "bath_slope1": s2c})
    df_rel["rel_error"] = 100.0 * (df_rel["slope1"] - df_rel["bath_slope1"]) / df_rel["bath_slope1"]
    df_rel["abs_rel_error"] = df_rel["rel_error"].abs()
    df_rel_u = df_rel.drop_duplicates(subset=["slope1", "bath_slope1"], keep="first")

    mean_rel = float(df_rel_u["rel_error"].mean())
    mean_abs = float(df_rel_u["abs_rel_error"].mean())
    p68      = float(np.percentile(df_rel_u["abs_rel_error"].values, 68))

    return {
        "pearson": pr, "pearson_p": prp,
        "spearman": sr, "spearman_p": srp,
        "mae": mae, "rmse": rmse, "r2": r2,
        "mean_rel": mean_rel, "mean_abs_rel": mean_abs, "p68_abs_rel": p68
    }

def _print_group_header(seg_id, g_sorted):
    """Header with first/last node_id and dist_to_out (p_dist_out)."""
    first_row = g_sorted.iloc[0]
    last_row  = g_sorted.iloc[-1]
    # assumes columns exist:
    first_node = first_row.get('node_id', 'NA')
    last_node  = last_row.get('node_id', 'NA')
    first_dist = float(first_row['p_dist_out'])
    last_dist  = float(last_row['p_dist_out'])
    print(f"\n=== Segment {int(seg_id)} ===")
    print(f"First node_id: {first_node}  |  Last node_id: {last_node}")
    print(f"First p_dist_out: {first_dist:.3f}  |  Last p_dist_out: {last_dist:.3f}")

def _plots_for_group(g_sorted, seg_id, save_prefix=None):
    """Your full plotting suite, scoped to the current group."""
    s1 = g_sorted['slope1'].values
    s2 = g_sorted['bath_slope1'].values
    x  = g_sorted['p_dist_out'].values

    # ---------- Relative error CDF (+ KDE inset) ----------
    rel_error = 100.0 * (s1 - s2) / s2
    abs_rel_error = np.abs(rel_error)
    df_rel = pd.DataFrame({"slope1": s1, "bath_slope1": s2,
                           "rel_error": rel_error, "abs_rel_error": abs_rel_error})
    df_rel_unique = df_rel.drop_duplicates(subset=['slope1','bath_slope1'], keep='first')

    bias_sorted = np.sort(df_rel_unique['rel_error'].values)
    cdf_bias = np.arange(1, len(bias_sorted)+1) / len(bias_sorted)
    abs_sorted = np.sort(df_rel_unique['abs_rel_error'].values)
    cdf_abs = np.arange(1, len(abs_sorted)+1) / len(abs_sorted)

    mean_bias = df_rel_unique['rel_error'].mean()
    mean_abs_rel_error = df_rel_unique['abs_rel_error'].mean()
    _, spearman_p = spearmanr(df_rel_unique['slope1'], df_rel_unique['bath_slope1'])
    p68 = np.percentile(df_rel_unique['abs_rel_error'], 68)

    color_bias = '#ff5722'
    color_abs  = '#fbc02d'
    color_p68  = 'dimgrey'
    color_zero = '#ffa726'
    color_mean = '#2979ff'

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_facecolor('#f5f5f5')
    ax.scatter(bias_sorted, cdf_bias, color=color_bias, label='Relative Error CDF (bias, %)',
               s=70, edgecolors='black', linewidth=0.5, marker='o', alpha=0.7)
    ax.scatter(abs_sorted, cdf_abs, color=color_abs, label='Absolute Relative Error CDF (%)',
               s=25, edgecolors='black', linewidth=0.5, marker='D', alpha=0.9)
    ax.axvline(x=0, color=color_zero, linestyle='--', linewidth=1.8, label='Zero Bias')
    ax.axvline(x=mean_bias, color=color_mean, linestyle='-.', linewidth=2, label=f'Mean Bias = {mean_bias:.2f} %')
    ax.axvline(x=p68, color=color_p68, linestyle=':', linewidth=2.5, label=f'|68%ile|: {p68:.2f} %')
    ax.axhline(y=0.68, color=color_p68, linestyle=':', linewidth=2.5)
    xmin = np.floor(min(bias_sorted.min(), abs_sorted.min())) - 1
    xmax = np.ceil(max(bias_sorted.max(), abs_sorted.max(), p68)) + 1
    ax.set_xlim(xmin, xmax)
    ax.set_xticks(np.arange(xmin, xmax + 1, 20))
    ax.set_xlabel('Relative Error (%)', fontsize=14)
    ax.set_ylabel('Cumulative Probability', fontsize=14)
    ax.set_title(f'Segment {int(seg_id)} – CDF of Slope Relative Error\n', fontsize=16, weight='bold')
    ax.grid(True, which='major', linestyle='--', alpha=0.5)
    ax.legend(fontsize=12, loc='lower right', frameon=True, fancybox=True)

    ax_kde = inset_axes(ax, width="100%", height="20%", loc='upper center',
                        bbox_to_anchor=(0, 1.05, 1, 0.3), bbox_transform=ax.transAxes, borderpad=0)
    sns.kdeplot(x=df_rel_unique['rel_error'], ax=ax_kde, fill=True, linewidth=2, alpha=0.3)
    ax_kde.set_xlim(xmin, xmax); ax_kde.axis('off')
    plt.tight_layout()
    if save_prefix: plt.savefig(f"{save_prefix}_seg{int(seg_id)}_cdf_rel_error.png", dpi=200)

    # ---------- Distance plot + boxplot ----------
    fig = plt.figure(figsize=(15,6), facecolor='white')
    gs  = GridSpec(1, 2, width_ratios=[5, 1.5], wspace=0.15)
    c1, c2 = '#1f77b4', '#ff7f0e'
    ax_main = fig.add_subplot(gs[0])
    ax_main.plot(x, s1, color=c1, linestyle='-', linewidth=1.5, marker='o', markersize=5,
                 markerfacecolor='white', markeredgecolor=c1, markeredgewidth=1.0, label='SWOT slope')
    ax_main.plot(x, s2, color=c2, linestyle='--', linewidth=1.5, marker='D', markersize=5,
                 markerfacecolor='white', markeredgecolor=c2, markeredgewidth=1.0, label='Bathymetry slope')
    ax_main.fill_between(x, s1, alpha=0.1, color=c1)
    ax_main.fill_between(x, s2, alpha=0.1, color=c2)
    ax_main.set_xlabel('Distance to outlet (m)', fontsize=12, weight='bold')
    ax_main.set_ylabel('Slope value', fontsize=12, weight='bold')
    ax_main.set_title(f'Segment {int(seg_id)} – Slopes vs Distance', fontsize=16, weight='bold')
    ax_main.grid(alpha=0.3, linestyle='--')
    ax_main.invert_xaxis()
    ax_main.legend(loc='upper left', frameon=True, facecolor='white', edgecolor='black')

    ax_box = fig.add_subplot(gs[1], sharey=ax_main)
    bp = ax_box.boxplot([s1, s2], positions=[1,2], widths=0.6, patch_artist=True, showfliers=False,
                        medianprops=dict(color='black'), whiskerprops=dict(color='black'), capprops=dict(color='black'))
    for patch, color in zip(bp['boxes'], [c1, c2]):
        patch.set_facecolor(color); patch.set_alpha(0.3)
    for i,(y,color) in enumerate([(s1,c1),(s2,c2)], start=1):
        xi = np.random.normal(i, 0.04, size=len(y))
        ax_box.scatter(xi, y, color=color, alpha=0.6, s=12, edgecolors='none', zorder=10)
    ax_box.set_xticks([1,2]); ax_box.set_xticklabels(['SWOT','Bathymetry'], fontsize=10, weight='bold')
    ax_box.margins(x=0.3); ax_box.yaxis.tick_right(); ax_box.yaxis.set_label_position("right")
    ax_box.grid(True, axis='y', linestyle='--', alpha=0.4)
    plt.tight_layout()
    if save_prefix: plt.savefig(f"{save_prefix}_seg{int(seg_id)}_slopes_vs_dist_box.png", dpi=200)

    # ---------- Scatter 1:1 ----------
    plt.figure(figsize=(6,6))
    plt.scatter(s1, s2, color='red', alpha=0.7, edgecolors='none', s=30)
    lims = [min(s1.min(), s2.min()), max(s1.max(), s2.max())]
    plt.plot(lims, lims, ls='--', lw=1.5, color='green', label='1:1 line')
    margin = 0.001
    low, high = lims
    plt.xlim(low - margin, high + margin); plt.ylim(low - margin, high + margin)
    plt.xlabel('SWOT slope', fontsize=12, weight='bold')
    plt.ylabel('Bathymetry slope', fontsize=12, weight='bold')
    plt.title(f'Segment {int(seg_id)} – SWOT vs Bath Slopes', fontsize=14, weight='bold')
    plt.grid(alpha=0.3, linestyle='--'); plt.legend(); plt.tight_layout()
    if save_prefix: plt.savefig(f"{save_prefix}_seg{int(seg_id)}_scatter_1to1.png", dpi=200)

    # ---------- Hexbin + BA + Residuals + CDF of |res| + Histograms ----------
    # hexbin
    plt.figure(figsize=(6,6))
    plt.hexbin(s1, s2, gridsize=100, mincnt=1, cmap='Blues')
    lims = [min(s1.min(), s2.min()), max(s1.max(), s2.max())]
    plt.plot(lims, lims, ls='--', lw=1.5, color='green', label='1:1 line')
    plt.xlim(lims); plt.ylim(lims)
    plt.xlabel('SWOT slope', fontsize=12, weight='bold')
    plt.ylabel('Bathymetry slope', fontsize=12, weight='bold')
    plt.title(f'Segment {int(seg_id)} – Hexbin SWOT vs Bath', fontsize=14, weight='bold')
    plt.legend(); plt.grid(alpha=0.3, linestyle='--'); plt.tight_layout()
    if save_prefix: plt.savefig(f"{save_prefix}_seg{int(seg_id)}_hexbin.png", dpi=200)

    # differences
    diff = s1 - s2
    avg  = 0.5 * (s1 + s2)
    mean_diff = diff.mean(); std_diff = diff.std()

    # Bland–Altman
    plt.figure(figsize=(6,4))
    plt.scatter(avg, diff, s=20, alpha=0.5)
    plt.axhline(mean_diff, color='red', linestyle='--', label='Mean diff')
    plt.axhline(mean_diff + 1.96*std_diff, color='gray', linestyle=':', label='±1.96σ')
    plt.axhline(mean_diff - 1.96*std_diff, color='gray', linestyle=':')
    plt.xlabel('Average slope', fontsize=12, weight='bold')
    plt.ylabel('Difference (SWOT – Bath)', fontsize=12, weight='bold')
    plt.title(f'Segment {int(seg_id)} – Bland–Altman', fontsize=14, weight='bold')
    plt.legend(); plt.grid(alpha=0.3, linestyle='--'); plt.tight_layout()
    if save_prefix: plt.savefig(f"{save_prefix}_seg{int(seg_id)}_bland_altman.png", dpi=200)

    # Residuals vs SWOT slope
    plt.figure(figsize=(6,4))
    plt.scatter(s1, diff, s=20, alpha=0.5)
    plt.axhline(0, color='black', linestyle='--', lw=1)
    plt.xlabel('SWOT slope', fontsize=12, weight='bold')
    plt.ylabel('Residual (SWOT – Bath)', fontsize=12, weight='bold')
    plt.title(f'Segment {int(seg_id)} – Residuals vs SWOT', fontsize=14, weight='bold')
    plt.grid(alpha=0.3, linestyle='--'); plt.tight_layout()
    if save_prefix: plt.savefig(f"{save_prefix}_seg{int(seg_id)}_residuals_vs_swot.png", dpi=200)

    # CDF of |res|
    abs_diff = np.abs(diff[np.isfinite(diff)])
    xs = np.sort(abs_diff); cdf = np.arange(1, len(xs)+1) / len(xs)
    p50 = np.percentile(abs_diff, 50); p68 = np.percentile(abs_diff, 68); p95 = np.percentile(abs_diff, 95)
    plt.figure(figsize=(6,4))
    plt.plot(xs, cdf, lw=2)
    plt.axvline(p68, linestyle='--', label=f'68%tile = {p68:.4f}')
    plt.axvline(p50, linestyle=':',  label=f'Median = {p50:.4f}')
    plt.axvline(p95, linestyle=':',  label=f'95%tile = {p95:.4f}')
    plt.xlabel('|Residual| (|SWOT – Bath|)', fontsize=12, weight='bold')
    plt.ylabel('Cumulative Probability', fontsize=12, weight='bold')
    plt.title(f'Segment {int(seg_id)} – CDF of |Residuals|', fontsize=14, weight='bold')
    plt.legend(); plt.grid(alpha=0.3, linestyle='--'); plt.tight_layout()
    if save_prefix: plt.savefig(f"{save_prefix}_seg{int(seg_id)}_cdf_abs_residuals.png", dpi=200)

    # Histogram + Violin
    plt.figure(figsize=(6,4))
    plt.hist(diff[np.isfinite(diff)], bins=30, density=True, alpha=0.7)
    plt.axvline(mean_diff, linestyle='--', label=f'Mean = {mean_diff:.4f}')
    plt.xlabel('Residual (SWOT – Bath)', fontsize=12, weight='bold')
    plt.ylabel('Density', fontsize=12, weight='bold')
    plt.title(f'Segment {int(seg_id)} – Residual Distribution', fontsize=14, weight='bold')
    plt.legend(); plt.grid(alpha=0.3, linestyle='--'); plt.tight_layout()
    if save_prefix: plt.savefig(f"{save_prefix}_seg{int(seg_id)}_hist_density.png", dpi=200)

    diff_clean = diff[np.isfinite(diff)]
    q1, med, q3 = np.percentile(diff_clean, [25, 50, 75])
    mean_val    = diff_clean.mean()
    p68v        = np.percentile(diff_clean, 68)
    ymin = np.percentile(diff_clean, 1); ymax = np.percentile(diff_clean, 99)
    pad  = 0.02 * (ymax - ymin); ylim = (ymin - pad, ymax + pad)
    plt.figure(figsize=(5,4))
    ax = sns.violinplot(y=diff_clean, inner='box', color='lightblue', cut=0, scale='width', bw=0.2, linewidth=1.2)
    ax.axhline(0, color='black', linestyle='--', lw=1)
    ax.scatter(0, mean_val, s=60, marker='D', color='#d62728', zorder=5, label=f"Mean = {mean_val:.4f}")
    ax.scatter(0, med,      s=60, marker='o', facecolors='white', edgecolors='black', zorder=5, label=f"Median = {med:.4f}")
    ax.scatter(0, q1,       s=80, marker='_', color='#2ca02c', zorder=5, label=f"Q1 = {q1:.4f}")
    ax.scatter(0, q3,       s=80, marker='_', color='#1f77b4', zorder=5, label=f"Q3 = {q3:.4f}")
    ax.axhline(p68v, color='purple', linestyle=':', lw=1.2, label=f"68%tile = {p68v:.4f}")
    ax.set_ylim(ylim); ax.set_ylabel('Residual (SWOT - Bath)', fontsize=12, weight='bold')
    ax.set_title(f'Segment {int(seg_id)} – Residual Distribution Around Zero', fontsize=14, weight='bold')
    ax.set_xticks([]); ax.legend(loc='lower left', fontsize=9, frameon=True)
    plt.tight_layout()
    if save_prefix: plt.savefig(f"{save_prefix}_seg{int(seg_id)}_violin_residuals.png", dpi=200)

def report_and_plot_consecutive_true_groups(annotated_df: pd.DataFrame, save_prefix=None):
    """
    For each consecutive TRUE group (segment_id), print the detailed metrics report
    and generate all plots for that group. Assumes annotated_df has:
    ['segment_id','flagged','node_id','p_dist_out','slope1','bath_slope1'].
    """
    # Ensure groups are contiguous by distance (your segmentation walked farthest→nearest)
    gdf = annotated_df.copy().sort_values('p_dist_out', ascending=False)
    # Iterate only over TRUE groups (segment_id not NaN)
    for seg_id in sorted(gdf['segment_id'].dropna().unique()):
        g = gdf[gdf['segment_id'] == seg_id].copy().sort_values('p_dist_out', ascending=False)
        _print_group_header(seg_id, g)

        stats = _stats_report_for_group(g)
        print(f"Pearson correlation: {stats['pearson']:.3f} (p={stats['pearson_p']:.2g})")
        print(f"Spearman correlation: {stats['spearman']:.3f} (p={stats['spearman_p']:.2g})")
        print(f"Mean Absolute Error (MAE): {stats['mae']:.4f}")
        print(f"Root Mean Squared Error (RMSE): {stats['rmse']:.4f}")
        print(f"R\u00b2: {stats['r2']:.3f}")
        print(f"Mean Relative Error (Bias): {stats['mean_rel']:.2f}%")
        print(f"Mean Absolute Relative Error: {stats['mean_abs_rel']:.2f}%")
        print(f"68th percentile Absolute Relative Error: {stats['p68_abs_rel']:.2f}%")

        # Plots for this group
        _plots_for_group(g, seg_id, save_prefix=save_prefix)

In [ ]:
report_and_plot_consecutive_true_groups(annotated_df, save_prefix="seg_report")

## Summary

In [ ]:
# Data
data = {
    'River': ['Cape Fear', 'Atrato', 'Sacramento', 'Pee Dee', 'Po', 'Garonne'],
    'r_40':  [0.44, 0.62, 0.64, 0.54, 0.21, 0.18],
    'ρ_40':  [0.42, 0.65, 0.67, 0.52, 0.30, 0.08],
    'r_80':  [0.41, 0.83, 0.45, 0.1, 0.34, 0.24],
    'ρ_80':  [0.37, 0.70, 0.51, -0.08, 0.36, 0.06],
}

df = pd.DataFrame(data)

# Reshape to long format
df_long = pd.DataFrame({
    'River': df['River'].tolist() * 2,
    'Width': ['40m'] * len(df) + ['80m'] * len(df),
    'r': df['r_40'].tolist() + df['r_80'].tolist(),
    'ρ': df['ρ_40'].tolist() + df['ρ_80'].tolist()
})

# Plot
fig, ax = plt.subplots(figsize=(8, 6))

# Scatter points by width
colors = {'40m': 'tab:blue', '80m': 'tab:orange'}
markers = {'40m': 'o', '80m': 's'}

for width in df_long['Width'].unique():
    subset = df_long[df_long['Width'] == width]
    ax.scatter(
        subset['r'],
        subset['ρ'],
        label=width,
        c=colors[width],
        marker=markers[width],
        s=100,
        edgecolor='black'
    )

# Annotate basins
for _, row in df_long.iterrows():
    ax.text(row['r'] + 0.01, row['ρ'] + 0.01, row['River'], fontsize=8)

# Labels and style
ax.set_xlabel('r', fontsize=12, fontweight='bold')
ax.set_ylabel('ρ', fontsize=12, fontweight='bold')
ax.set_title('Scatter Plot of r vs ρ by Segment Width', fontsize=14, fontweight='bold')
ax.axhline(0, color='gray', linestyle='--', linewidth=0.8)
ax.axvline(0, color='gray', linestyle='--', linewidth=0.8)
ax.legend(title='Segment Width')
ax.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.colors import TwoSlopeNorm
from matplotlib.lines import Line2D
from matplotlib.collections import LineCollection
# =========================
# Config
# =========================
threshold = 0.40
outline_color = 'limegreen'
dash_on, dash_off = 1.5, 2.0  # dotted look
lw = 2

# =========================
# Data
# =========================
data = {
    'River': ['Cape Fear', 'Atrato', 'Sacramento', 'Pee Dee', 'Po', 'Garonne'],
    'r_40':  [0.44, 0.62, 0.64, 0.54, 0.21, 0.18],
    'ρ_40':  [0.42, 0.65, 0.67, 0.52, 0.30, 0.08],
    'r_80':  [0.41, 0.83, 0.45, 0.1, 0.34, 0.24],
    'ρ_80':  [0.37, 0.70, 0.51, -0.08, 0.36, 0.06],
}
df = pd.DataFrame(data).set_index('River')
df = df[['r_40', 'ρ_40', 'r_80', 'ρ_80']]

# =========================
# Heatmap
# =========================
vals = df.values
vmax = np.nanmax(np.abs(vals))
norm = TwoSlopeNorm(vmin=-vmax, vcenter=0.0, vmax=vmax)

fig, ax = plt.subplots(figsize=(7.8, 5.0))
im = ax.imshow(vals, aspect='auto', cmap='RdBu_r', norm=norm, origin='upper')

ax.set_xticks(np.arange(df.shape[1]))
ax.set_xticklabels(df.columns, fontsize=11, fontweight='bold')
ax.set_yticks(np.arange(df.shape[0]))
ax.set_yticklabels(df.index, fontsize=11)

# annotate values
for i in range(vals.shape[0]):
    for j in range(vals.shape[1]):
        ax.text(j, i, f"{vals[i, j]:.2f}",
                ha='center', va='center', fontsize=9,
                color='black' if abs(vals[i, j]) < vmax*0.55 else 'white')

ax.set_title('Correlation Heatmap (centered at 0)', fontsize=13, fontweight='bold', pad=10)
ax.set_xlabel('metric@Segment Width', fontsize=11, fontweight='bold')
ax.set_ylabel('River', fontsize=11, fontweight='bold')
ax.set_frame_on(False)

cbar = fig.colorbar(im, ax=ax, shrink=0.9, pad=0.02)
cbar.set_label('Correlation', fontsize=11, fontweight='bold')

# =========================
# Boxy perimeter for r > threshold OR ρ > threshold
# =========================
mask = vals > threshold  # includes both Pearson and Spearman

rows, cols = mask.shape
edges = set()

def add_edge(p1, p2):
    edges.add((p1, p2) if p1 < p2 else (p2, p1))

for i in range(rows):
    for j in range(cols):
        if not mask[i, j]:
            continue
        x0, x1 = j - 0.5, j + 0.5
        y0, y1 = i - 0.5, i + 0.5
        if i == 0 or not mask[i-1, j]:      # top
            add_edge((x0, y0), (x1, y0))
        if i == rows-1 or not mask[i+1, j]: # bottom
            add_edge((x0, y1), (x1, y1))
        if j == 0 or not mask[i, j-1]:      # left
            add_edge((x0, y0), (x0, y1))
        if j == cols-1 or not mask[i, j+1]: # right
            add_edge((x1, y0), (x1, y1))

segments = [list(edge) for edge in edges]

if segments:
    lc = LineCollection(
        segments,
        colors=outline_color,
        linewidths=3,
        linestyles=(0, (dash_on, dash_off)),
        capstyle='butt',
        joinstyle='miter',
        zorder=3,
    )
    ax.add_collection(lc)

# keep full pixel grid visible
ax.set_xlim(-0.5, cols - 0.5)
ax.set_ylim(rows - 0.5, -0.5)

# =========================
# Legend outside the axes
# =========================
outline_handle = Line2D([0], [0], color=outline_color, lw=lw, linestyle=(0, (dash_on, dash_off)))
fig.legend([outline_handle], [f'Pearson or Spearman > {threshold:.2f}'],
           loc='lower right', bbox_to_anchor=(0.98, -0.05), frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
# ==== INPUT DATA (replace with your real numbers) ====
data = {
    'River': ['Cape Fear', 'Atrato', 'Sacramento', 'Pee Dee', 'Po', 'Garonne'],
    # 40 m
    'bias_40':  [-77.2,  53.4, -29.9,  -41.4, -13.6, -77.1],   # %
    'p68_40':   [87.7, 99.6, 55.9, 66.3,  66.3,  86.2],  # %
    # 80 m
    'bias_80':  [ -78.3, 19.1, -18.6, -16.3,  -4.14,  -78.6],  # %
    'p68_80':   [88.0, 51.27, 50.5, 77.7,  56.3,  86.46],  # %
}
df = pd.DataFrame(data)

# ==== PLOT ====
fig, ax = plt.subplots(figsize=(11, 6))

rivers = df['River'].values
x = np.arange(len(rivers))              # river positions
group_w = 0.36                          # width of each width-group (40m vs 80m)
bar_w   = group_w / 2.0                 # two bars inside each group (Bias, P68)

# Offsets: 40m group to the left, 80m group to the right
x40 = x - group_w/2
x80 = x + group_w/2

# Colors per metric (consistent across widths)
colors = {'Bias': '#4361ee', '68%ile': '#f59e0b'}

# 40 m bars
b1 = ax.bar(x40 - bar_w/2, df['bias_40'], width=bar_w, label='Bias (40 m)', color=colors['Bias'])
b2 = ax.bar(x40 + bar_w/2, df['p68_40'],  width=bar_w, label='68%ile (40 m)', color=colors['68%ile'])

# 80 m bars
b3 = ax.bar(x80 - bar_w/2, df['bias_80'], width=bar_w, label='Bias (80 m)', color=colors['Bias'], alpha=0.55)
b4 = ax.bar(x80 + bar_w/2, df['p68_80'],  width=bar_w, label='68%ile (80 m)', color=colors['68%ile'], alpha=0.55)

# Zero line (helps read positive/negative bias)
ax.axhline(0, color='gray', linewidth=1, linestyle='--', zorder=0)

# X labels
ax.set_xticks(x)
ax.set_xticklabels(rivers, rotation=30, ha='right')

ax.set_ylabel('Percent (%)', fontweight='bold')
ax.set_title('Bias and 68th-Percentile Error by River and Segment Width', fontweight='bold')

# Legend (metric color + width opacity cue)
leg = ax.legend(ncol=2, frameon=False, loc='upper center')

# Value labels on bars
def label_bars(containers):
    for c in containers:
        for rect in c:
            h = rect.get_height()
            ax.annotate(f'{h:.0f}%',
                        xy=(rect.get_x() + rect.get_width()/2, h),
                        xytext=(0, 4 if h>=0 else -12),
                        textcoords='offset points',
                        ha='center', va='bottom' if h>=0 else 'top',
                        fontsize=8)
label_bars([b1, b2, b3, b4])

# Tidy layout
ymin = min(df[['bias_40','bias_80']].min().min() - 10, -10)
ymax = max(df[['p68_40','p68_80']].max().max() + 10,  10)
ax.set_ylim(ymin, ymax)
ax.grid(axis='y', linestyle=':', alpha=0.35)

plt.tight_layout()
plt.show()


In [ ]:
# ===== DATA (your example) =====
data = {
    'River':  ['Cape Fear','Atrato','Sacramento','Pee Dee','Po','Garonne'],
    'bias_40':[-77.2, 53.4, -29.9, -41.4, -13.6, -77.1],   # %
    'p68_40': [ 87.7, 99.6,  55.9,  66.3,  66.3,  86.2],   # %
    'bias_80':[-78.3, 19.1, -18.6, -16.3,  -4.14, -78.6],  # %
    'p68_80': [ 88.0, 51.27, 50.5,  77.7,  56.3,  86.46],  # %
}
df = pd.DataFrame(data)

# OPTIONAL: sort rivers for a tidy visual (pick one metric to sort on)
# df = df.sort_values('bias_40').reset_index(drop=True)

# Prepare matrices
rivers = df['River'].tolist()
bias = df[['bias_40','bias_80']].to_numpy()
p68  = df[['p68_40','p68_80']].to_numpy()

# ===== PLOT =====
fig, (ax1, ax2) = plt.subplots(
    2, 1, figsize=(8.8, 7.2),
    sharex=True, sharey=True,
    gridspec_kw={'height_ratios': [1, 1], 'hspace': 0.15}
)

# ---- Heatmap 1: Bias (centered at 0) ----
bmax = np.nanmax(np.abs(bias))
b_norm = TwoSlopeNorm(vmin=-bmax, vcenter=0.0, vmax=bmax)
im1 = ax1.imshow(bias, cmap='RdBu_r', norm=b_norm, aspect='auto', origin='upper')

# annotate bias
for i in range(bias.shape[0]):
    for j in range(bias.shape[1]):
        ax1.text(j, i, f"{bias[i, j]:.1f}",
                 ha='center', va='center',
                 color='black' if abs(bias[i, j]) < 0.55*bmax else 'white',
                 fontsize=9)

ax1.set_title('Bias (%)', fontweight='bold', pad=8)
ax1.set_yticks(np.arange(len(rivers)))
ax1.set_yticklabels(rivers)
ax1.set_xticks([0, 1])
ax1.set_xticklabels(['40 m', '80 m'], fontweight='bold')
ax1.set_ylabel('River', fontweight='bold')
cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.046, pad=0.02)
cbar1.set_label('Bias (%)', fontweight='bold')

# ---- Heatmap 2: 68%ile error ----
e_min, e_max = np.nanmin(p68), np.nanmax(p68)
im2 = ax2.imshow(p68, cmap='Oranges', vmin=e_min, vmax=e_max, aspect='auto', origin='upper')

# annotate p68
for i in range(p68.shape[0]):
    for j in range(p68.shape[1]):
        ax2.text(j, i, f"{p68[i, j]:.1f}",
                 ha='center', va='center',
                 color='black' if p68[i, j] < 0.6*e_max else 'white',
                 fontsize=9)

ax2.set_title('68th Percentile Error (%)', fontweight='bold', pad=8)
ax2.set_yticks(np.arange(len(rivers)))
ax2.set_yticklabels(rivers)  # sharey keeps alignment
ax2.set_xticks([0, 1])
ax2.set_xticklabels(['40 m', '80 m'], fontweight='bold')
ax2.set_xlabel('Segment Width', fontweight='bold')
ax2.set_ylabel('River', fontweight='bold')
cbar2 = fig.colorbar(im2, ax=ax2, fraction=0.046, pad=0.02)
cbar2.set_label('Absolute Relative Error (%)', fontweight='bold')

# cosmetic gridlines around cells (optional)
for ax in (ax1, ax2):
    ax.set_xticks(np.arange(-0.5, 2, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, len(rivers), 1), minor=True)
    ax.grid(which='minor', color='white', linestyle='-', linewidth=1.0)
    ax.tick_params(which='minor', bottom=False, left=False)

fig.suptitle('Dual Heatmaps: Bias and 68%ile Error by River & Width', fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

In [ ]:
# ----- DATA -----
data = {
    'River':  ['Cape Fear','Atrato','Sacramento','Pee Dee','Po','Garonne'],
    'bias_40':[-77.2, 53.4, -29.9, -41.4, -13.6, -77.1],
    'p68_40': [ 87.7, 99.6,  55.9,  66.3,  66.3,  86.2],
    'bias_80':[-78.3, 19.1, -18.6, -16.3,  -4.14, -78.6],
    'p68_80': [ 88.0, 51.27, 50.5,  77.7,  56.3,  86.46],
}
df = pd.DataFrame(data)

# Long format
long = pd.DataFrame({
    'River': df['River'].tolist()*2,
    'Width': ['40 m']*len(df) + ['80 m']*len(df),
    'Bias':  df['bias_40'].tolist() + df['bias_80'].tolist(),
    'P68':   df['p68_40'].tolist()  + df['p68_80'].tolist()
})

# Plot
fig, ax = plt.subplots(figsize=(8.6, 6.4))

for width, subset in long.groupby('Width'):
    ax.scatter(subset['Bias'], subset['P68'], s=120, alpha=0.9, label=width, edgecolor='black')
    # label each point with the river (optional)
    for _, r in subset.iterrows():
        ax.text(r['Bias']+1.5, r['P68']+1.0, r['River'], fontsize=8)

ax.axvline(0, linestyle='--', linewidth=0.8)
ax.set_xlabel('Bias (%)', fontweight='bold')
ax.set_ylabel('68th Percentile Error (%)', fontweight='bold')
ax.set_title('Bias vs 68%ile Error — Bubble Plot', fontweight='bold')
ax.grid(True, linestyle=':', alpha=0.35)
ax.legend(frameon=False, title='Segment Width')
plt.tight_layout(); plt.show()


## Width variability

In [ ]:
df = pd.read_csv(path2w_wse)
cv_width = df.groupby('node_id')['width'].agg(lambda x: x.std()/x.mean()).rename("CV_width")

In [ ]:
mean_cv = cv_width.mean()
print(mean_cv)

In [ ]:
CV_rivers ={
    'River':  ["Cape Fear", "Atrato", "Sacramento", "Pee Dee", "Po", "Garonne"],
    'cv_val':      [0.16, 0.39, 0.30, 0.29, 0.2, 0.08],
    'ρ_10%':       [0.47, 0.56, 0.35, 0.70, 0.29, 0.18],
    'ρ_15%':       [0.38, 0.59, 0.44, 0.63,0.32, 0.2],
    'ρ_20%':       [0.35, 0.54, 0.57, 0.57, 0.36, 0.19],
    'ρ_23%':       [0.44, 0.60 ,0.53, 0.33, 0.38, 0.19],
    'ρ_25%':       [0.44, 0.59, 0.57, 0.29, 0.39, 0.19],
    'ρ_30%':       [0.44, 0.63, 0.69, 0.17, 0.39,0.19],
    'bias(%) 10%': [-66.51, 64.85, -22.37, -58.53, -10.94, -81.62],
    'bias(%) 15%': [-61.56, 59.45, -23.35, -44.77, -3.23, -81.95],
    'bias(%) 20%': [-70.95, 54.26, -26.67, -38.62, 0.12, -83.11],
    'bias(%) 23%': [-70.95, 32.19, -31.52, -25.26, 1.77, -82.62],
    'bias(%) 25%': [-70.95, 43.42, -30.00, -13.78, 1.60, -82.62],
    'bias(%) 30%': [-70.95, 27.68, -28.89, -3.96, 14.07, -82.62],
    '68"%"ile 10%': [81.11, 103.96, 66.13, 71.83, 67.65, 89.78],
    '68"%"ile 15%': [81.11, 99.08, 67.36, 64.00, 61.75, 89.12],
    '68"%"ile 20%': [81.91, 95.11, 69.54, 56.02, 59.94, 89.63],
    '68"%"ile 23%': [81.91, 73.08, 58.52, 58.14, 56.04, 89.63],
    '68"%"ile 25%': [81.91, 91.57, 56.98, 73.22, 52.98, 89.63],
    '68"%"ile 30%': [81.91, 81.39, 53.38, 76.25, 58.63, 89.63],
}

summary_cv = pd.DataFrame(CV_rivers)

In [ ]:
from matplotlib.ticker import MultipleLocator, PercentFormatter
# Columns for panels
rho_cols = summary_cv.filter(regex=r'^68"%\"?ile\s*\d+%$').columns.tolist()

# ---- Per-river label offsets (pixels) to prevent overlap ----
LABEL_OFFSET = {
    "Cape Fear": (-10, 10),
    "Po":        (10, 10),   # push down so it won't collide with Cape Fear
    "Sacramento":(10, 10),
    "Atrato":    (10, 10),
    "Pee Dee":   (-10, 10),
    "Garonne":   (10, -4),
}

# ---- Plot ----
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for ax, col in zip(axes, rho_cols):
    x = summary_cv['cv_val']
    y = summary_cv[col]

    # Scatter (zorder so points under labels)
    ax.scatter(x, y, s=100, zorder=2)

    # River labels with per-river offsets + leader lines
    for xi, yi, name in zip(x, y, summary_cv['River']):
        dx, dy = LABEL_OFFSET.get(name, (10, 6))
        ax.annotate(
            name,
            xy=(xi, yi),
            xytext=(dx, dy),
            textcoords='offset points',
            ha='left' if dx >= 0 else 'right',
            va='center',
            fontsize=12,
            bbox=None,
            arrowprops=dict(arrowstyle='-', lw=0.7, alpha=0.7),
            zorder=0,
            clip_on=False
        )

    # Fixed scales
    ax.set_xlim(0, 0.5)
    ax.set_ylim(0, 105)

    # Force y-axis ticks every 0.1
    ax.yaxis.set_major_locator(MultipleLocator(20))
    ax.yaxis.set_major_formatter(PercentFormatter())

    # Bigger tick labels
    ax.tick_params(axis='both', which='major', labelsize=14)

    # Axis labels
    ax.set_xlabel('CV', fontsize=14)
    ax.set_ylabel(col, fontsize=14)

    # Inner dashed grid (major + minor)
    ax.grid(True, which='major', linestyle='--', linewidth=0.8, alpha=0.5)
    ax.minorticks_on()
    ax.grid(True, which='minor', linestyle=':', linewidth=0.6, alpha=0.35)

    # Spearman ρ inside plot (bottom-right)
    r, _ = spearmanr(x, y)
    ax.text(0.95, 0.05, f"ρ = {r:.2f}",
            transform=ax.transAxes, ha='right', va='bottom',
            fontsize=10, fontweight='bold')

# Hide unused panels if any (not needed here but safe)
for ax in axes[len(rho_cols):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Plot
plt.figure(figsize=(7,5))
plt.scatter(summary["CV_width"], summary["median_rho"], s=100, color="steelblue")

# Add labels for each point
for i, row in summary.iterrows():
    plt.text(row["CV_width"]+0.005, row["median_rho"], row["riverID"], fontsize=9)

plt.xlabel("Width Variability (CV)", fontsize=12)
plt.ylabel("Median Spearman's ρ", fontsize=12)
plt.title("Relationship between Width Variability and Spearman Correlation", fontsize=13)
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()

## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
from matplotlib.lines import Line2D
def plot_river_grid(df, river_order=None, suptitle="r vs ρ across rivers"):
    required = {"river", "setting", "r", "rho", "r_p", "rho_p"}
    if missing := required - set(df.columns):
        raise ValueError(f"Missing columns: {missing}")

    rivers = list(pd.unique(df["river"]))
    if river_order:
        rivers = [r for r in river_order if r in rivers] + [r for r in pd.unique(df["river"]) if r not in river_order]
    rivers = rivers[:6]

    # Color map per setting
    setting_order = list(pd.unique(df["setting"].astype(str)))
    cmap = plt.cm.get_cmap("tab10", max(1, len(setting_order)))
    color_map = {s: cmap(i) for i, s in enumerate(setting_order)}

    # Global axis limits
    xmin = min(0.0, df["r"].min()) - 0.02
    xmax = max(1.0, df["r"].max()) + 0.02
    ymin = min(0.0, df["rho"].min()) - 0.02
    ymax = max(1.0, df["rho"].max()) + 0.02

    # Grid
    fig, axes = plt.subplots(2, 3, figsize=(13, 7), dpi=140, sharex=True, sharey=True)
    axes = axes.ravel()

    setting_handles = {}

    for ax, river in zip(axes, rivers):
        sub = df[df["river"] == river]

        for setting, chunk in sub.groupby("setting"):
            col = color_map[setting]
            for _, row in chunk.iterrows():
                nonsig = (row["r_p"] > 0.05) or (row["rho_p"] > 0.05)
                marker = "x" if nonsig else "o"
                kwargs = dict(s=70, marker=marker)
                if nonsig:
                    kwargs.update(color=col, linewidth=1.0, alpha=0.95)
                else:
                    kwargs.update(facecolor=col, edgecolor="black", linewidth=0.6, alpha=0.95)
                ax.scatter(row["r"], row["rho"], **kwargs)

            if setting not in setting_handles:
                setting_handles[setting] = Line2D(
                    [], [], marker='o', linestyle='None',
                    markerfacecolor=col, markeredgecolor='black',
                    markersize=7, label=str(setting)
                )

        ax.set_title(str(river), fontsize=11)
        ax.grid(True, linestyle=":", alpha=0.35)
        ax.axvline(0, lw=0.6, alpha=0.4)
        ax.axhline(0, lw=0.6, alpha=0.4)
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)

    for i, ax in enumerate(axes):
        if i % 3 == 0:
            ax.set_ylabel("ρ (Spearman)")
        if i >= 3:
            ax.set_xlabel("r (Pearson)")

    for j in range(len(rivers), 6):
        axes[j].set_visible(False)

    # Legends outside
    settings_legend = fig.legend(
        handles=[setting_handles[s] for s in setting_order if s in setting_handles],
        labels=[s for s in setting_order if s in setting_handles],
        title="Segment width",
        loc="lower center",
        ncol=min(5, len(setting_handles)),
        frameon=True,
        bbox_to_anchor=(0.5, -0.12)   # OUTSIDE bottom
    )

    signif_handles = [
        Line2D([], [], marker='o', linestyle='None',
               markerfacecolor='white', markeredgecolor='black',
               markersize=7, markeredgewidth=0.8, label='p ≤ 0.05 (significant)'),
        Line2D([], [], marker='x', linestyle='None',
               color='black', markersize=7, label='p > 0.05 (not significant)')
    ]
    fig.legend(
        handles=signif_handles,
        loc="center left",
        frameon=True,
        title="Significance",
        bbox_to_anchor=(1.01, 0.5)   # OUTSIDE right
    )

    fig.suptitle(suptitle, y=0.99, fontsize=14)
    plt.tight_layout(rect=[0.02, 0.05, 0.9, 0.95])  # leave room
    plt.show()

In [ ]:
df = pd.DataFrame([
     {"river":"Cape Fear","setting":"40 m","r":0.44,"r_p":0.04,"rho":0.42,"rho_p":0.05},
     {"river":"Cape Fear","setting":"80 m","r":0.41,"r_p":0.06,"rho":0.37,"rho_p":0.09},
     {"river":"Cape Fear","setting":"10%","r":0.53,"r_p":0.02,"rho":0.47,"rho_p":0.04},
     {"river":"Cape Fear","setting":"15%","r":0.44,"r_p":0.06,"rho":0.38,"rho_p":0.1},
     {"river":"Cape Fear","setting":"20%","r":0.35,"r_p":0.15,"rho":0.44,"rho_p":0.06},
     {"river":"Cape Fear","setting":"23%","r":0.35,"r_p":0.15,"rho":0.44,"rho_p":0.06},
     {"river":"Cape Fear","setting":"25%","r":0.35,"r_p":0.15,"rho":0.44,"rho_p":0.06},
     {"river":"Cape Fear","setting":"30%","r":0.35,"r_p":0.15,"rho":0.44,"rho_p":0.06},
     {"river":"Atrato","setting":"40 m","r":0.62,"r_p":2.5e-4,"rho":0.65,"rho_p":9.3e-5},
     {"river":"Atrato","setting":"80 m","r":0.83,"r_p":4.3e-8,"rho":0.70,"rho_p":3e-5},
     {"river":"Atrato","setting":"10%","r":0.35,"r_p":0.07,"rho":0.56,"rho_p":0.003},
     {"river":"Atrato","setting":"15%","r":0.60,"r_p":0.002,"rho":0.59,"rho_p":0.002},
     {"river":"Atrato","setting":"20%","r":0.49,"r_p":0.009,"rho":0.54,"rho_p":0.003},
     {"river":"Atrato","setting":"25%","r":0.46,"r_p":0.013,"rho":0.59,"rho_p":0.001},
     {"river":"Atrato","setting":"30%","r":0.55,"r_p":0.003,"rho":0.63,"rho_p":4.7e-4},
     {"river":"Sacramento","setting":"40 m","r":0.64,"r_p":4.6e-6,"rho":0.67,"rho_p":1e-6},
     {"river":"Sacramento","setting":"80 m","r":0.45,"r_p":0.003,"rho":0.51,"rho_p":5e-4},
     {"river":"Sacramento","setting":"10%","r":0.37,"r_p":0.02,"rho":0.35,"rho_p":0.02},
     {"river":"Sacramento","setting":"15%","r":0.46,"r_p":0.002,"rho":0.44,"rho_p":0.004},
     {"river":"Sacramento","setting":"20%","r":0.56,"r_p":1.2e-4,"rho":0.57,"rho_p":7.1e-5},
     {"river":"Sacramento","setting":"23%","r":0.54,"r_p":2.6e-4,"rho":0.53,"rho_p":4.2e-4},
     {"river":"Sacramento","setting":"25%","r":0.58,"r_p":4e-5,"rho":0.57,"rho_p":7.5e-5},
     {"river":"Sacramento","setting":"30%","r":0.65,"r_p":2.5e-6,"rho":0.69,"rho_p":3.8e-7},
     {"river":"Pee Dee","setting":"40 m","r":0.54,"r_p":1.7e-4,"rho":0.52,"rho_p":3.4e-4},
     {"river":"Pee Dee","setting":"80 m","r":0.10,"r_p":0.52,"rho":-0.08,"rho_p":0.59},
     {"river":"Pee Dee","setting":"10%","r":0.63,"r_p":3.3e-6,"rho":0.7,"rho_p":9.7e-8},
     {"river":"Pee Dee","setting":"15%","r":0.69,"r_p":4.1e-7,"rho":0.63,"rho_p":6.6e-6},
     {"river":"Pee Dee","setting":"20%","r":0.65,"r_p":2.4e-6,"rho":0.57,"rho_p":6.8e-5},
     {"river":"Pee Dee","setting":"23%","r":0.41,"r_p":0.006,"rho":0.33,"rho_p":0.03},
     {"river":"Pee Dee","setting":"25%","r":0.32,"r_p":0.03,"rho":0.29,"rho_p":0.056},
     {"river":"Pee Dee","setting":"30%","r":0.14,"r_p":0.4,"rho":0.17,"rho_p":0.25},
     {"river":"Po","setting":"40 m","r":0.21,"r_p":0.004,"rho":0.3,"rho_p":2.8e-5},
     {"river":"Po","setting":"80 m","r":0.34,"r_p":1.6e-6,"rho":0.36,"rho_p":3.6e-7},
     {"river":"Po","setting":"10%","r":0.2,"r_p":0.005,"rho":0.29,"rho_p":4e-5},
     {"river":"Po","setting":"15%","r":0.28,"r_p":6.4e-5,"rho":0.32,"rho_p":4.8e-6},
     {"river":"Po","setting":"20%","r":0.34,"r_p":1.3e-6,"rho":0.36,"rho_p":3.2e-7},
     {"river":"Po","setting":"23%","r":0.36,"r_p":5e-7,"rho":0.38,"rho_p":9.4e-8},
     {"river":"Po","setting":"25%","r":0.36,"r_p":2.7e-7,"rho":0.39,"rho_p":2.3e-8},
     {"river":"Po","setting":"30%","r":0.34,"r_p":2.1e-6,"rho":0.39,"rho_p":3.6e-8},
     {"river":"Garonne","setting":"40 m","r":0.18,"r_p":0.46,"rho":0.08,"rho_p":0.74},
     {"river":"Garonne","setting":"80 m","r":0.24,"r_p":0.34,"rho":0.06,"rho_p":0.81},
     {"river":"Garonne","setting":"10%","r":0.53,"r_p":0.04,"rho":0.42,"rho_p":0.1},
     {"river":"Garonne","setting":"15%","r":0.4,"r_p":0.14,"rho":0.41,"rho_p":0.12},
     {"river":"Garonne","setting":"20%","r":0.35,"r_p":0.16,"rho":0.26,"rho_p":0.32},
     {"river":"Garonne","setting":"23%","r":0.26,"r_p":0.3,"rho":0.23,"rho_p":0.37},
     {"river":"Garonne","setting":"25%","r":0.26,"r_p":0.31,"rho":0.16,"rho_p":0.54},
     {"river":"Garonne","setting":"30%","r":0.18,"r_p":0.47,"rho":0.09,"rho_p":0.73},
      
])
plot_river_grid(df, river_order=["Cape Fear","Atrato","Sacramento","Pee Dee","Po","Garonne"])


In [ ]:
def plot_bias_p68_grid(
    df,
    river_order=None,
    suptitle="Bias vs 68%ile Error across rivers",
    bias_col="bias_pct",     # x-axis (e.g., -78.3 means -78.3%)
    p68_col="p68_pct",       # y-axis (e.g., 88.0 means 88%)
):
    """
    Faceted 2x3 scatter: one subplot per river with shared axes.
    Expects tidy df columns: river, setting, bias_col, p68_col.

    - Color encodes 'setting'
    - Vertical dashed line at Bias = 0
    - Legends placed OUTSIDE the grid (no overlap)
    """

    required = {"river", "setting", bias_col, p68_col}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    # Stable river order
    rivers = list(pd.unique(df["river"]))
    if river_order:
        rivers = [r for r in river_order if r in rivers] + [r for r in rivers if r not in river_order]
    rivers = rivers[:6]

    # Palette per setting
    setting_order = list(pd.unique(df["setting"].astype(str)))
    cmap = plt.cm.get_cmap("tab10", max(1, len(setting_order)))
    color_map = {s: cmap(i) for i, s in enumerate(setting_order)}

    # Global limits (include 0 on x for reference)
    xmin = min(0.0, df[bias_col].min()) - 2
    xmax = max(0.0, df[bias_col].max()) + 2
    ymin = max(0.0, min(df[p68_col].min(), 0)) - 2
    ymax = df[p68_col].max() + 2

    fig, axes = plt.subplots(2, 3, figsize=(13, 7), dpi=140, sharex=True, sharey=True)
    axes = axes.ravel()

    # One proxy per setting for legend
    setting_handles = {}

    for ax, river in zip(axes, rivers):
        sub = df[df["river"] == river]

        for setting, chunk in sub.groupby("setting"):
            col = color_map[setting]
            ax.scatter(
                chunk[bias_col], chunk[p68_col],
                s=70, marker="o", facecolor=col, edgecolor="black", linewidth=0.6, alpha=0.95
            )
            if setting not in setting_handles:
                setting_handles[setting] = Line2D(
                    [], [], marker='o', linestyle='None',
                    markerfacecolor=col, markeredgecolor='black',
                    markersize=7, label=str(setting)
                )

        # Cosmetics
        ax.set_title(str(river), fontsize=11)
        ax.grid(True, linestyle=":", alpha=0.35)
        ax.axvline(0, lw=1.0, alpha=0.5, linestyle="--")  # Bias = 0 reference
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)

    # Label only left/bottom axes
    for i, ax in enumerate(axes):
        if i % 3 == 0:
            ax.set_ylabel("68th Percentile Error (%)")
        if i >= 3:
            ax.set_xlabel("Bias (%)")

    # Hide unused axes if <6 rivers
    for j in range(len(rivers), 6):
        axes[j].set_visible(False)

    # Settings legend OUTSIDE bottom
    fig.legend(
        handles=[setting_handles[s] for s in setting_order if s in setting_handles],
        labels=[s for s in setting_order if s in setting_handles],
        title="Setting",
        loc="lower center",
        ncol=min(6, len(setting_handles)),
        frameon=True,
        bbox_to_anchor=(0.5, -0.12)
    )

    fig.suptitle(suptitle, y=0.99, fontsize=14)
    plt.tight_layout(rect=[0.02, 0.05, 0.98, 0.95])  # room for legend & title
    plt.show()

In [ ]:
df = pd.DataFrame([
     {"river":"Cape Fear","setting":"40 m","bias_pct":-77.2,"p68_pct": 87.7},
     {"river":"Cape Fear","setting":"80 m","bias_pct":-78.3,"p68_pct": 88.0},
     {"river":"Cape Fear","setting":"10%","bias_pct":-66.5,"p68_pct": 81.1},
     {"river":"Cape Fear","setting":"15%","bias_pct":-61.6,"p68_pct": 81.1},
     {"river":"Cape Fear","setting":"20%","bias_pct":-70.9,"p68_pct": 81.9},
     {"river":"Cape Fear","setting":"23%","bias_pct":-70.9,"p68_pct": 81.9},
     {"river":"Cape Fear","setting":"25%","bias_pct":-70.9,"p68_pct": 81.9},
     {"river":"Cape Fear","setting":"30%","bias_pct":-70.9,"p68_pct": 81.9},
     {"river":"Atrato","setting":"40 m","bias_pct":53.4,"p68_pct": 99.6},
     {"river":"Atrato","setting":"80 m","bias_pct":19.1,"p68_pct": 51.3},
     {"river":"Atrato","setting":"10%","bias_pct":64.9,"p68_pct": 103.9},
     {"river":"Atrato","setting":"15%","bias_pct":59.5,"p68_pct": 99.1},
     {"river":"Atrato","setting":"20%","bias_pct":54.3,"p68_pct": 95.1},
     {"river":"Atrato","setting":"25%","bias_pct":43.4,"p68_pct": 91.6},
     {"river":"Atrato","setting":"30%","bias_pct":27.7,"p68_pct": 81.4},
     {"river":"Sacramento","setting":"40 m","bias_pct":-29.9,"p68_pct": 55.9},
     {"river":"Sacramento","setting":"80 m","bias_pct":-18.6,"p68_pct": 50.5},
     {"river":"Sacramento","setting":"10%","bias_pct":-22.4,"p68_pct": 66.13},
     {"river":"Sacramento","setting":"15%","bias_pct":-23.4,"p68_pct": 67.4},
     {"river":"Sacramento","setting":"20%","bias_pct":-26.7,"p68_pct": 69.5},
     {"river":"Sacramento","setting":"23%","bias_pct":-31.5,"p68_pct": 58.5},
     {"river":"Sacramento","setting":"25%","bias_pct":-30.0,"p68_pct": 57.0},
     {"river":"Sacramento","setting":"30%","bias_pct":-28.9,"p68_pct": 53.4},
     {"river":"Pee Dee","setting":"40 m","bias_pct":-41.4,"p68_pct": 66.3},
     {"river":"Pee Dee","setting":"80 m","bias_pct":-16.3,"p68_pct": 77.7},
     {"river":"Pee Dee","setting":"10%","bias_pct":-58.5,"p68_pct": 71.8},
     {"river":"Pee Dee","setting":"15%","bias_pct":-44.8,"p68_pct": 50.5},
     {"river":"Pee Dee","setting":"20%","bias_pct":-38.6,"p68_pct": 56.0},
     {"river":"Pee Dee","setting":"23%","bias_pct":-25.3,"p68_pct": 58.1},
     {"river":"Pee Dee","setting":"25%","bias_pct":-13.8,"p68_pct": 73.2},
     {"river":"Pee Dee","setting":"30%","bias_pct":-3.96,"p68_pct": 76.6},
     {"river":"Po","setting":"40 m","bias_pct":-13.6,"p68_pct": 66.3},
     {"river":"Po","setting":"80 m","bias_pct":-4.14,"p68_pct": 56.3},
     {"river":"Po","setting":"10%","bias_pct":-10.9,"p68_pct": 67.7},
     {"river":"Po","setting":"15%","bias_pct":-3.23,"p68_pct": 61.8},
     {"river":"Po","setting":"20%","bias_pct":0.12,"p68_pct": 59.9},
     {"river":"Po","setting":"23%","bias_pct":1.8,"p68_pct": 56.0},
     {"river":"Po","setting":"25%","bias_pct":1.6,"p68_pct": 53.0},
     {"river":"Po","setting":"30%","bias_pct":14.1,"p68_pct": 58.6},
     {"river":"Garonne","setting":"40 m","bias_pct":-77.9,"p68_pct": 86.2},
     {"river":"Garonne","setting":"80 m","bias_pct":-78.6,"p68_pct": 86.5},
     {"river":"Garonne","setting":"10%","bias_pct":-85.1,"p68_pct": 89.1},
     {"river":"Garonne","setting":"15%","bias_pct":-84.2,"p68_pct": 87.4},
     {"river":"Garonne","setting":"20%","bias_pct":-82.0,"p68_pct": 86.8},
     {"river":"Garonne","setting":"23%","bias_pct":-80.5,"p68_pct": 86.5},
     {"river":"Garonne","setting":"25%","bias_pct":-79.9,"p68_pct": 86.5},
     {"river":"Garonne","setting":"30%","bias_pct":-78.1,"p68_pct": 86.2},

])

In [ ]:
plot_bias_p68_grid(df, river_order=["Cape Fear","Atrato","Sacramento","Pee Dee","Po","Garonne"])